- This is the submission notebook where we will demonstrate different Actions performed in Trilink powered by BigQuery AI
- You need to setup the datasets required for this process first suing the create_datasets and create_solution_embeddings first

In [ ]:
import os
import sys
from pathlib import Path
# Add project root to sys.path
project_root = Path().resolve().parent  # one level up from notebooks/
sys.path.append(str(project_root))

import time
from datetime import datetime

from src.calltranscripts_simulation import EnhancedCallCenterDataGenerator

import pandas as pd
from google.cloud import bigquery
from pandas_gbq import to_gbq
from pprint import pprint

# Initialize BigQuery client
client = bigquery.Client(project='trilink-472019')

##Set the project ID and database - Change to your project ID and database
project_id='trilink-472019'
database = 'database'

# Load the extension
%load_ext google.cloud.bigquery
%load_ext bigquery_magics

#### Pre Steps - Let's simulate the call data for month of september (300 calls) - Already done in the data creation notebook

- We will use this simulated call data for September to demo all the actions like:
    1. Follow up/Next best action email for calls that require one
    2. Outbound calls to make with call script for upsell or cross sell
    3. Weekly Agent Performance Report for Supervisor with instructions of future training for each agent
    4. Agent Weekly Personalized Training Material
- This is added as a sperate sim process and append to calldata to recreate exactly how it would have happened in the business in real scenario
- The different actions we perform, would be daily,weekly or monthly and hence we do the separate call data simulation for the current/test month

##### Append the calls to bigquery call and add embeddings

In [4]:
# ##Append the calls to bigquery call table
# calls_df.to_gbq(
#     destination_table='trilink-472019.database.call_transcripts_raw',
#     project_id='trilink-472019',
#     if_exists='append',
#     credentials=None  # Uses default credentials
# )

- We can also add the embeddings for the calls
    - We will add the embeddings for new calls and append to existing embedding table at the end of each day after the followup email is processed

In [5]:
# %%bigquery
# INSERT INTO `trilink-471315.database.call_transcripts_raw_embeddings`
# SELECT 
#   raw.*,
#   (SELECT ml_generate_embedding_result 
#    FROM ML.GENERATE_EMBEDDING(
#      MODEL `trilink-471315.database.text_embedding_005_model`,
#      (SELECT raw.call_transcript AS content),
#      STRUCT(TRUE AS flatten_json_output)
#    )
#   ) AS call_transcript_embedding
# FROM `trilink-471315.database.call_transcripts_raw` raw
# LEFT JOIN `trilink-471315.database.call_transcripts_raw_embeddings` emb
#   ON raw.call_id = emb.call_id
# WHERE emb.call_id IS NULL

#### Action 1 - Create Follow Up/Next Best action Email

- Idea is to create a personalized follow-up/Next Best action Email for all customers who called the within 24 hours of their call
    - We first decide looking at the call transcript if the call needs a follow up email using AI.GENERATE_BOOL for calls in the period of interest (Demo done for 2025-09-15)
    - For the calls that needs an email, we get their embeddings
    - For each calls we will do vector search of the call embeddings with all the call transcript embeddings to find similar calls where the agent had a successful solution/conversation with customer. We do this by adding metadata filtering of overall_satisfaction > 6 and the most similar calls. These calls are then used for reference to add additional solutions for the customer which the agent might have missed
    - For each calls we also do the vector search to the business problem solution table created to find the most similar problem and solution already mentioned by business. This could be used to find the solution that business offers in a similar situation and can help in adding more actionable items in followup email
    - We will then get the customer and service info of all the products the customers who needs followup email
    - Then we we will create the Hyper personalized Email with followup/next best action as primary objective and cross/upsell as secondary option for the customers with information like below in the prompt:-
        - customer details
        - internet plan details and status
        - mobile plan details and status
        - Home security product status and devices bought if any
        - call transcript for which we are creating the follow up email
        - similar call for additional solution reference
        - Upsell/ Cross Sell Product Options to picth, if it seems reasonable to  
    - The whole customers have the email generated and is written to the followup email table, which can be triggered as a daily job to deliver the emails to each customer

In [6]:
%%bigquery
##Dates for which you want to run the follow up email
DECLARE cdate1 DATE DEFAULT DATE('2025-09-15');
DECLARE cdate2 DATE DEFAULT DATE('2025-09-15');

CREATE OR REPLACE TABLE `trilink-472019.database.follow_up_emails` AS (
    WITH target_calls AS (
  SELECT 
    call_id,
    customer_id,
    call_transcript,
    call_date,
    AI.GENERATE_BOOL(
      CONCAT(
        'Analyze this customer service call transcript and determine if it requires email follow-up. ',
        'We want to err on the side of caution - SEND EMAIL FOLLOW-UP for ANY call that is NOT purely routine. ',
        '',
        'SEND EMAIL FOLLOW-UP if the call involves ANY of these: ',
        '1. Customer expressed ANY dissatisfaction, frustration, confusion, or unhappiness ',
        '2. ANY technical issues, problems, complaints, or concerns were mentioned ',
        '3. ANY solutions, explanations, or troubleshooting steps were provided ',
        '4. Customer asked questions about products, services, or account details ',
        '5. ANY account changes, modifications, or requests were discussed ',
        '6. Customer seemed uncertain or needed clarification on anything ',
        '7. ANY follow-up actions, callbacks, or next steps were mentioned ',
        '8. Opportunities for additional services or products were identified ',
        '9. Customer tone suggested they were not completely satisfied ',
        '10. Call involved anything beyond the most basic routine transactions. ',
        '11. Call that says there is a possibility of upsell/cross sell to the customer'
        '',
        'ONLY skip email follow-up for these very limited routine cases: ',
        '- Simple payment made with confirmed success and happy customer ',
        '- Quick balance inquiry with no issues or questions ',
        '- Brief, successful IVR navigation with satisfied customer ',
        '- Standard account verification completed smoothly with no other discussion. ',
        '',
        'When in doubt, choose TRUE for email follow-up. Call transcript: "', call_transcript, '"'
      ),
      connection_id => 'us.vertex-ai-connection',
      endpoint => 'gemini-2.5-flash'
    ).result as email_needed
    
  FROM `trilink-472019.database.call_transcripts_raw`
  WHERE PARSE_DATE('%Y-%m-%d', call_date) >= cdate1
    AND PARSE_DATE('%Y-%m-%d', call_date) <= cdate2
),

email_calls_with_embeddings AS (
  SELECT *
  FROM ML.GENERATE_EMBEDDING(
    MODEL `trilink-472019.database.text_embedding_005_model`,
    (SELECT 
       call_transcript AS content,
       call_id,
       customer_id,
       call_date,
       email_needed
     FROM target_calls 
     WHERE email_needed = TRUE),
    STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_DOCUMENT' AS task_type)
  )
),

similar_email_added as (
  SELECT query.call_id,
  query.customer_id,
  query.call_date,
  query.content,base.call_transcript,base.primary_scenario,base.overall_rating,vs.distance
FROM
  VECTOR_SEARCH(
    TABLE `trilink-472019.database.call_transcripts_raw_embeddings`,
    'call_transcript_embedding',
    TABLE email_calls_with_embeddings,
    'ml_generate_embedding_result',
    top_k => 10,
    distance_type => 'COSINE') vs
  WHERE CAST(vs.base.overall_rating AS INT64) > 6
  
),

similar_calls_added AS (
  SELECT 
    call_id,
    customer_id,
    call_date,
    content as current_call_transcript,
    
    -- Concatenate all similar call transcripts and ratings
    STRING_AGG(
      CONCAT(
        'Rating: ', overall_rating, '/10\n',
        'Scenario: ', primary_scenario, '\n', 
        'Call: ', call_transcript, '\n\n---\n\n'
      )
      ORDER BY CAST(overall_rating AS INT64) DESC, distance ASC
    ) as all_similar_calls
    
  FROM similar_email_added
  GROUP BY call_id, customer_id, call_date, content
),

-- Aggregate security devices per customer
security_aggregated AS (
  SELECT 
    customer_id,
    STRING_AGG(device_type, ', ' ORDER BY device_type) AS device_types_concat,
    COUNT(device_type) AS total_security_devices
  FROM `trilink-472019.database.security_df`
  GROUP BY customer_id
),

--Add the best available solution for the call form the business solution search
solution_base as 
(
  SELECT 
    vs.base.problem AS similar_matched_problem,
    vs.base.solution AS recommended_solution,
    vs.base.sector AS sector,
    vs.query.call_id as call_id,
    vs.distance
  FROM VECTOR_SEARCH(
    TABLE `trilink-472019.database.business_solution_embeddings`,
    'problem_embedding',
      TABLE email_calls_with_embeddings,
    'ml_generate_embedding_result',
    top_k => 1,
    distance_type => 'COSINE'
  ) AS vs
),


--Get the current customer services and products and possibility for corrsell or upsell
base AS (
  SELECT 
    a.*,
    e.age,
    e.city,
    e.family_size,
    e.fiber_availability,
    e.home_ownership,
    e.home_type,
    e.life_stage,
    e.neighborhood_crime_rate,
    e.work_from_home_flag,
    b.device_types_concat,
    b.total_security_devices,
    c.speed_mbps AS internet_speed,
    c.plan_tier AS internet_plan,
    c.contract_type AS internet_contract_type,
    c.internet_tenure_days,
    d.plan_type AS mobile_plan,
    d.contract_type AS mobile_contract_type,
    d.data_overage_frequency AS mobile_data_overage_frequency,
    d.monthly_cost AS mobile_monthly_cost,
    f.similar_matched_problem,
    f.recommended_solution,
    f.sector,
    
    -- Customer type flags
    CASE WHEN b.device_types_concat IS NULL THEN 0 ELSE 1 END AS security_customer,
    CASE WHEN c.internet_churn IS NULL THEN 'Never been our internet customer'
      WHEN c.internet_churn = 1 THEN 'Churned internet customer'
      ELSE 'Active internet customer' END AS internet_customer_status,
    CASE WHEN d.mobile_churn IS NULL THEN 'Never been our mobile customer'
      WHEN d.mobile_churn = 1 THEN 'Churned mobile customer'
      ELSE 'Active mobile customer'
    END AS mobile_customer_status
    
  FROM similar_calls_added a
  LEFT JOIN security_aggregated b ON a.customer_id = b.customer_id
  LEFT JOIN `trilink-472019.database.customer_df` e ON a.customer_id = e.customer_id
  LEFT JOIN `trilink-472019.database.internet_df` c ON a.customer_id = c.customer_id AND c.internet_churn = 0
  LEFT JOIN `trilink-472019.database.mobile_df` d ON a.customer_id = d.customer_id AND d.mobile_churn = 0
  LEFT JOIN solution_base f on a.call_id = f.call_id
)

SELECT 
  call_id,
  customer_id,
  call_date,
  current_call_transcript,
  all_similar_calls,
  similar_matched_problem,
  recommended_solution,
  
  AI.GENERATE(
    CONCAT(
      'Create a personalized follow-up/next best action for solving customers issues email based on this customer call and examples of successful agent approaches.\\n\\n',
      
      '=== CUSTOMER INFO ===\\n',
      'Customer ID: ', customer_id, '\\n',
      'Customer Age: ', age, '\\n',
      'Customer City: ', city, '\\n',
      'Customer Family Size: ', family_size, '\\n',
      'Customer Fiber Availability: ', CASE WHEN fiber_availability THEN 'Yes' ELSE 'No' END, '\\n',
      'Customer Home Ownership: ', home_ownership, '\\n',
      'Customer Home Type: ', home_type, '\\n',
      'Customer Life Stage: ', life_stage, '\\n',
      'Customer Neighborhood Crime Rate: ', neighborhood_crime_rate, '\\n',
      'Customer Work From Home Flag: ', CASE WHEN work_from_home_flag THEN 'Yes' ELSE 'No' END, '\\n\\n',
          
      '=== CUSTOMER CURRENT SERVICES ===\\n',
      'Internet Status: ', internet_customer_status, 
      CASE WHEN internet_customer_status = 'Active internet customer' 
        THEN CONCAT(' (Plan: ', COALESCE(internet_plan, 'Unknown'), ', Speed: ', COALESCE(CAST(internet_speed AS STRING), 'Unknown'), ' Mbps)')
        ELSE '' END, '\\n',
      'Mobile Status: ', mobile_customer_status,
      CASE WHEN mobile_customer_status = 'Active mobile customer'
        THEN CONCAT(' (Plan: ', COALESCE(mobile_plan, 'Unknown'), ', Monthly Cost: $', COALESCE(CAST(mobile_monthly_cost AS STRING), 'Unknown'), ')')
        ELSE '' END, '\\n',
      'Security Services: ', 
      CASE WHEN security_customer = 1 
        THEN CONCAT('Yes (Devices: ', COALESCE(device_types_concat, 'Unknown'), ', Total: ', COALESCE(CAST(total_security_devices AS STRING), '0'), ')')
        ELSE 'No security services' END, '\\n\\n',

      '=== CURRENT CALL DETAILS ===\\n',
      'Date: ', call_date, '\\n',
      'Call ID: ', call_id, '\\n',
      'Call Transcript: ', current_call_transcript, '\\n\\n',

      '=== Business Solution ===\\n'
      'From the business solution guidelines with potential solutions you can use to solve different problems, here is the most similar problem to what customers is\\n',
      'Similar Problem:',similar_matched_problem,'\\n', 
      'Business Solution to the problem:',recommended_solution,'\\n',
      'You may use this solution to add any additional solutions that could have been provided for the customer in the follow up email\\n', 

      
      '=== AVAILABLE UPGRADE/CROSS-SELL OPTIONS ===\\n',
      'INTERNET PLANS AVAILABLE:\\n',
      '• Premium Gig: 1000 Mbps at $100/month\\n',
      '• Standard: 100 Mbps at $70/month\\n', 
      '• Basic: 25 Mbps at $40/month\\n\\n',
      
      'MOBILE PLANS AVAILABLE:\\n',
      '• Limited 2GB: $30/month (Canada only, international $1/min)\\n',
      '• Limited 5GB: $40/month (Canada only, international $1/min)\\n',
      '• Limited 10GB: $50/month (Canada only, international $1/min)\\n',
      '• Unlimited Standard: $70/month (Free North America calls, 7-day international roaming)\\n',
      '• Unlimited Premium: $90/month (Free worldwide calls, 7-day international roaming, Free Disney Plus for 1 year)\\n\\n',
      
      'SECURITY DEVICES AVAILABLE:\\n',
      '• Motion Sensor: $30 each\\n',
      '• Window Sensor: $30 each\\n',
      '• Smart Doorbell: $80 (optional $10/month monitoring)\\n',
      '• Smart Lock: $60\\n',
      '• Indoor Camera: $120 (audio communication +$30)\\n',
      '• Outdoor Camera: $220 (optional $10/month monitoring, includes 1-year free Spotlight Pro)\\n',
      '• Security Panel: $250 (requires $30/month monitoring for alarm setup)\\n\\n',
      
      '=== SIMILAR SUCCESSFUL APPROACHES ===\\n',
      'If the current call transcript shows a pending solution, you may reference how other agents handled similar cases:\\n',
      COALESCE(all_similar_calls, 'No similar high-rated examples found'), '\\n\\n',
      
      '=== EMAIL INSTRUCTIONS ===\\n',
      'PRIMARY GOAL: Address the call transcript and provide appropriate follow-up/next best action for solving customers issues\\n',
      'SECONDARY GOAL (if appropriate): Suggest relevant upgrades/cross-sells that adds business value\\n\\n',
      
      'CROSS-SELL/UPSELL GUIDELINES:\\n',
      '• ONLY suggest services they dont currently have or clear upgrades\\n',
      '• Focus on how it could solve problems mentioned in the call or improve the customers current or future needs\\n',
      '• Keep suggestions brief - maximum 2-4 sentences\\n',
      '• Make it sound helpful, not sales-y. Add some small offers (none greater than 15$ for upsells/cross sells)\\n',
      'Mention what the upsell/cross sell product is, what it offers, what plan it is(if its a mobile or internet one) and how it would be great fit for the customer'
      '• If customer was upset/angry, DO NOT include any cross or upsell sales suggestions\\n\\n',
      
      'EMAIL REQUIREMENTS:\\n',
      '• Start with warm, professional greeting\\n',
      '• Reference the specific conversation and any issues discussed\\n',
      '• Provide clear next steps or resolution\\n',
      '• If appropriate, briefly mention how additional services could help\\n',
      '• End with contact information for further assistance\\n',
      '• Sign as "TriLink Customer Success Team"\\n',
      '• Keep tone personal and human, not automated\\n',
      '• Total length should be concise but thorough\\n',
      'Add the cross-sell and upsell options as long as it follows the guidelines',
        'For upsell/cross sell cases customer could reach out to 1800-TRILINK'
        
        
      '''****NOTE****: 
      Write the email with natural, personalized language. 
      Avoid placeholder text like [Customer Name],[Customers Full Name], [Company], 
      or other bracketed variables that make emails appear automated. If the customer's name is unknown, use 'Dear Valued Customer' as the greeting.
      
      If the call transcript have a placeholder name for the customer, igonore that name. Use Dear Valued Customer.
      
      IMPORTANT: The email should read as if it was written specifically for this recipient by a human, not generated from a template or an AI.'''  

    ),
    connection_id => 'us.vertex-ai-connection',
    endpoint => 'gemini-2.5-flash'
  ).result AS personalized_email,
  age,city,family_size,fiber_availability,home_ownership,home_type,life_stage,neighborhood_crime_rate,work_from_home_flag,device_types_concat,total_security_devices,internet_speed,internet_plan,internet_contract_type,internet_tenure_days,mobile_plan,mobile_contract_type,mobile_data_overage_frequency,mobile_monthly_cost,security_customer

FROM base);


Query is running:   0%|          |

""


In [7]:
%%bigquery followup_df
select * from `trilink-472019.database.follow_up_emails`;

Query is running:   0%|          |

Downloading:   0%|          |

In [8]:
print(f"Fallback Email created :{followup_df.shape[0]}")
followup_df.head(5)

Fallback Email created :9


,call_id,customer_id,call_date,current_call_transcript,all_similar_calls,similar_matched_problem,recommended_solution,personalized_email,age,city,...,total_security_devices,internet_speed,internet_plan,internet_contract_type,internet_tenure_days,mobile_plan,mobile_contract_type,mobile_data_overage_frequency,mobile_monthly_cost,security_customer
0,CALL_000015,C00043356,2025-09-15,**Call Transcript - TriLink Telecom Customer S...,Rating: 7/10\nScenario: payment_assistance\nCa...,Customer experiencing payment difficulties due...,Enroll in hardship assistance program with fle...,"Dear Alex Chen,\n\nThank you for speaking with...",35,Toronto,...,3,<NA>,None,None,NaN,Unlimited_Standard,12_Month,0,172,1
1,CALL_000233,C00087472,2025-09-15,**TriLink Telecom Customer Service Call Transc...,Rating: 7/10\nScenario: upsell_security_device...,Customer experiencing frequent outages lasting...,Install redundant connection with automatic fa...,"Dear Mr. Henderson,\n\nThank you for speaking ...",52,Waterloo,...,<NA>,<NA>,None,None,NaN,Unlimited_Standard,24_Month,0,233,0
2,CALL_000075,C00083289,2025-09-15,**TriLink Telecom Customer Service Call Transc...,Rating: 7/10\nScenario: complaint_resolution\n...,Customer with Limited_2GB plan hitting data li...,Upgrade to Unlimited_Standard $69/month elimin...,Subject: Following Up on Your Recent Call Rega...,54,London,...,<NA>,<NA>,None,None,NaN,Unlimited_Standard,24_Month,0,114,0
3,CALL_000199,C00028069,2025-09-15,**TriLink Telecom Customer Service Call Transc...,Rating: 7/10\nScenario: payment_assistance\nCa...,Customer with Limited_2GB plan hitting data li...,Upgrade to Unlimited_Standard $69/month elimin...,"Dear Jessica Miller,\n\nThank you for speaking...",29,London,...,<NA>,<NA>,None,None,NaN,Limited_10GB,Month_to_Month,5,95,0
4,CALL_000270,C00050583,2025-09-15,**TriLink Telecom Customer Service Call Transc...,Rating: 8/10\nScenario: equipment_replacement\...,Senior customer paying $85/month for premium p...,Downgrade to senior-friendly Limited_5GB plan ...,Subject: Following Up on Your Internet Service...,50,Toronto,...,3,1000,Premium_Gig,24_Month,679.0,None,None,<NA>,<NA>,1


In [9]:
import numpy as np
def display_cases(df_display, i=None):
    """
    Display customer call case information including transcript and follow-up email.
    
    Parameters:
    df_display (DataFrame): DataFrame containing customer call data
    i (int, optional): Index of the case to display. If None, selects a random case.
    """
    
    if i is None:
        # Fix: np.random.randint instead of np.rint for random integer selection
        i = np.random.randint(0, df_display.shape[0])
    
    # Ensure index is within bounds
    if i < 0 or i >= df_display.shape[0]:
        print(f"Error: Index {i} is out of bounds. DataFrame has {df_display.shape[0]} rows.")
        return
    
    print(f"Customer: {df_display.iloc[i]['customer_id']} called on {df_display.iloc[i]['call_date']}:")
    print("=" * 93)
    print("Hyperpersonalized Follow Up Email Generated:")
    print("=" * 93)
    pprint(df_display.iloc[i]['personalized_email'])
    print("=" * 93)
    print("Similar problems and Business solution for it:")
    print("=" * 93)
    pprint(df_display.iloc[i]['similar_matched_problem'])
    print("=" * 93)
    pprint(df_display.iloc[i]['recommended_solution'])
    print("=" * 93)
    print("Call Transcript:")
    print("=" * 93)
    pprint(df_display.iloc[i]['current_call_transcript'])
    print("=" * 93)

In [10]:
display_cases(followup_df,5)

Customer: C00091638 called on 2025-09-15:
Hyperpersonalized Follow Up Email Generated:
('Subject: Following Up on Your Internet Service Concerns - TriLink Telecom '
 '(Account C00091638)\n'
 '\n'
 'Dear Mr. Davis,\n'
 '\n'
 'This email is a follow-up to your call with our escalation specialist, '
 'Carlos Martinez, today, September 15th, regarding the intermittent speed '
 "issues you've been experiencing with your Premium Gig internet service.\n"
 '\n'
 "We understand how incredibly frustrating it is when your internet isn't "
 'performing as expected, especially when it impacts your work from home and '
 "your family's daily activities. Please accept our sincere apologies for the "
 "inconvenience and the unsatisfactory experience you've had, particularly "
 'after your previous call.\n'
 '\n'
 "As discussed with Carlos, we've scheduled a senior field technician to visit "
 'your home to thoroughly investigate the root cause of these persistent '
 'issues. This technician will be abl

In [11]:
display_cases(followup_df,8)

Customer: C00000643 called on 2025-09-15:
Hyperpersonalized Follow Up Email Generated:
('Subject: Following Up on Your TriLink Telecom Service Concerns - Case '
 'C00000643\n'
 '\n'
 'Dear Ms. Evelyn Reed,\n'
 '\n'
 'I hope this email finds you well.\n'
 '\n'
 'This message is a follow-up to your call with our Escalation Specialist, '
 'Carlos Martinez, on September 15th regarding the ongoing internet speed '
 'issues and your overall service experience with TriLink Telecom. We '
 "understand you've been incredibly frustrated with the recurring problems, "
 "and we sincerely apologize for the inconvenience and the time you've spent "
 'trying to resolve this.\n'
 '\n'
 "Carlos has detailed the steps we're taking to ensure a lasting resolution:\n"
 '\n'
 '1.  **Technician Visit Confirmed:** A field technician has been scheduled to '
 'visit your home on **Thursday, September 18th, between 1:00 PM and 5:00 '
 'PM**. Carlos has added detailed notes to the work order emphasizing the '
 'hi

#### Action 2 - Weekly Agent Performance Evaluation

- Idea is to create comprehensive weekly performance reports for all customer service agents to identify strengths, improvement areas, and provide coaching recommendations
    - We first extract all calls for each agent within the evaluation week (Demo done for 2025-01-20 to 2025-01-25)
    - For all calls in the evaluation period, we generate embeddings using the text embedding model
    - For each agent's call, we perform vector search against all historical call transcript embeddings to find the top 3 most similar calls from OTHER agents with high performance (overall_rating > 6). This creates a benchmark of successful approaches for similar customer scenarios
    - For each calls we also do the vector search to the business problem solution table created to find the most similar problem and solution already mentioned by business that agent ideally should implement
    - We then analyze each individual call using AI to identify:
        - Positive behaviors the agent demonstrated (professional communication, problem-solving, empathy, product knowledge)
        - Areas where the agent could improve (missed opportunities, communication gaps, process issues)
        - Specific better approaches the agent could have used based on the successful similar calls from other agents
    - We aggregate all individual call analyses by agent to create weekly summaries with information like:
        - Total calls handled, average rating, success rate
        - All positive behaviors from the week (consolidated into key strengths)
        - All improvement areas from the week (consolidated into development opportunities)  
        - All coaching suggestions from successful examples (consolidated into actionable recommendations)
    - Finally, we generate three AI-powered consolidated reports per agent:
        - **Weekly Positives**: 3-5 bullet points highlighting consistent strengths and good behaviors
        - **Weekly Negatives**: 3-5 bullet points identifying patterns of improvement opportunities
        - **Coaching Recommendations**: Specific coaching points, recommended phrases, skills to develop, and techniques for different scenarios
    - The complete agent performance data is written to the weekly_agent_performance table, which can be used by managers for constructive performance reviews and targeted coaching sessions

In [ ]:
%%bigquery
--Declare the week date variable
DECLARE eval_week_start DATE DEFAULT DATE('2025-09-14');
DECLARE eval_week_end DATE DEFAULT DATE('2025-09-20');

CREATE OR REPLACE TABLE `trilink-472019.database.weekly_agent_performance` AS (
  WITH weekly_calls AS (
    SELECT 
      agent_id,
      agent_name,
      call_id,
      customer_id,
      call_transcript,
      overall_rating,
      call_successful,
      customer_monthly_spend,
      customer_issue_history,
      call_date,
      primary_scenario
    FROM `trilink-472019.database.call_transcripts_raw`
    WHERE PARSE_DATE('%Y-%m-%d', call_date) >= eval_week_start
      AND PARSE_DATE('%Y-%m-%d', call_date) <= eval_week_end
  ),

  -- Generate embeddings for each call in the evaluation week
  weekly_calls_with_embeddings AS (
    SELECT *
    FROM ML.GENERATE_EMBEDDING(
      MODEL `trilink-472019.database.text_embedding_005_model`,
      (SELECT 
         call_transcript AS content,
         agent_id,
         agent_name,
         call_id,
         customer_id,
         overall_rating,
         call_successful,
         customer_monthly_spend,
         customer_issue_history,
         call_date,
         primary_scenario
       FROM weekly_calls),
      STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_DOCUMENT' AS task_type)
    )
  ),

  -- Find similar high-performing calls for each weekly call (LEFT JOIN to preserve all records)
similar_high_performance_calls AS (
  SELECT 
    wc.agent_id,
    wc.agent_name, 
    wc.call_id,
    wc.customer_id,
    wc.call_date,
    wc.call_transcript as current_call_transcript,
    wc.overall_rating as current_rating,
    wc.call_successful as current_success,
    wc.primary_scenario as current_scenario,
    
    -- Get similar calls (will be NULL if none found)
    STRING_AGG(
      CONCAT(
        'Agent: ', vs.base.agent_name, ' (ID: ', vs.base.agent_id, ')\n',
        'Rating: ', vs.base.overall_rating, '/10\n',
        'Success: ', CASE WHEN CAST(vs.base.call_successful AS STRING) = 'true' THEN 'Yes' ELSE 'No' END, '\n',
        'Scenario: ', vs.base.primary_scenario, '\n',
        'Call Transcript: ', vs.base.call_transcript, '\n\n---\n\n'
      )
      ORDER BY CAST(vs.base.overall_rating AS INT64) DESC, vs.distance ASC
      LIMIT 3
    ) as top_similar_calls
    
  FROM weekly_calls wc  -- Start with ALL weekly calls
  LEFT JOIN (           -- LEFT JOIN to preserve all
    SELECT *
    FROM VECTOR_SEARCH(
      TABLE `trilink-472019.database.call_transcripts_raw_embeddings`,
      'call_transcript_embedding', 
      TABLE weekly_calls_with_embeddings,
      'ml_generate_embedding_result',
      top_k => 10,
      distance_type => 'COSINE'
    )
    WHERE CAST(base.overall_rating AS INT64) > 6
      AND base.agent_id != query.agent_id
  ) vs ON wc.call_id = vs.query.call_id
  GROUP BY 
    wc.agent_id, wc.agent_name, wc.call_id, wc.customer_id, 
    wc.call_date, wc.call_transcript, wc.overall_rating, 
    wc.call_successful, wc.primary_scenario
),

--Add the best available solution for the call form the business solution search
solution_base as 
(
  SELECT 
    vs.base.problem AS similar_matched_problem,
    vs.base.solution AS recommended_solution,
    vs.base.sector AS sector,
    vs.query.call_id as call_id,
    vs.distance
  FROM VECTOR_SEARCH(
    TABLE `trilink-472019.database.business_solution_embeddings`,
    'problem_embedding',
      TABLE weekly_calls_with_embeddings,
      'ml_generate_embedding_result',
    top_k => 1,
    distance_type => 'COSINE'
  ) AS vs
),

base as 
(
  select a.*,b.similar_matched_problem,b.recommended_solution
   from similar_high_performance_calls a 
  left join solution_base b 
  on a.call_id = b.call_id
),

  -- Analyze each individual call for positives, negatives, and improvements
  individual_call_analysis AS (
    SELECT 
      agent_id,
      agent_name, 
      call_id,
      customer_id,
      call_date,
      current_call_transcript,
      current_rating,
      current_success,
      current_scenario,
      top_similar_calls,
      
      -- Analyze positive agent behaviors
      AI.GENERATE(
        CONCAT(
          'Analyze this customer service call transcript and identify ALL positive things the agent did well. ',
          'Focus on realistic agent behaviors within their control and job scope. ',
          'Look for: professional communication, problem-solving attempts, empathy, product knowledge, ',
          'following procedures, offering alternatives, setting expectations, active listening, etc.\n\n',
          
          'Call Details:\n',
          'Agent: ', agent_name, ' (ID: ', agent_id, ')\n',
          'Rating: ', current_rating, '/10\n',
          'Success: ', CASE WHEN CAST(current_success AS STRING) = 'true' THEN 'Yes' ELSE 'No' END, '\n',
          'Scenario: ', current_scenario, '\n\n',
          
          'Call Transcript:\n', current_call_transcript, '\n\n',
          
          'List ONLY the positive things this agent did well in short bullet points. ',
          'If no significant positives, respond with "No notable positives identified."'
        ),
        connection_id => 'us.vertex-ai-connection',
        endpoint => 'gemini-2.5-flash'
      ).result AS call_positives,
      
      -- Analyze negative/improvement areas
      AI.GENERATE(
        CONCAT(
          'Analyze this customer service call transcript and identify areas where the agent could improve. ',
          'Focus on realistic, actionable feedback within agent control. Be constructive, not punitive. ',
          'Look for: missed opportunities, communication gaps, process issues, knowledge gaps, ',
          'empathy lapses, not offering alternatives, unclear explanations, etc.\n\n',
          
          'Call Details:\n',
          'Agent: ', agent_name, ' (ID: ', agent_id, ')\n', 
          'Rating: ', current_rating, '/10\n',
          'Success: ', CASE WHEN CAST(current_success AS STRING) = 'true' THEN 'Yes' ELSE 'No' END, '\n',
          'Scenario: ', current_scenario, '\n\n',
          
          'Call Transcript:\n', current_call_transcript, '\n\n',
          
          'List improvement areas in short bullet points focusing on what agent could control. ',
          'If no significant issues, respond with "No notable improvement areas identified."'
        ),
        connection_id => 'us.vertex-ai-connection',
        endpoint => 'gemini-2.5-flash'
      ).result AS call_negatives,
      
      -- Suggest better approaches based on similar successful calls
      AI.GENERATE(
        CONCAT(
          'Based on this agents call and examples of similar successful calls by other agents, ',
          'suggest specific things this agent could have done better to improve the outcome.\n\n',
          
          'CURRENT AGENT CALL:\n',
          'Agent: ', agent_name, ' (ID: ', agent_id, ')\n',
          'Rating: ', current_rating, '/10\n', 
          'Success: ', CASE WHEN CAST(current_success AS STRING) = 'true' THEN 'Yes' ELSE 'No' END, '\n',
          'Scenario: ', current_scenario, '\n',
          'Transcript: ', current_call_transcript, '\n\n',

                '=== Business Solution ===\\n'
      'From the business solution guidelines with potential solutions you can use to solve different problems, here is the most similar problem to what customers is\\n',
      'Similar Problem:',similar_matched_problem,'\\n', 
      'Business Solution to the problem:',recommended_solution,'\\n',
      'You may use this solution to add any additional solutions that could have been provided for the customer in the follow up email\\n', 

          
          'EXAMPLES OF SUCCESSFUL SIMILAR CALLS:\n',
          COALESCE(top_similar_calls, 'No similar high-rated examples found'), '\n\n',
          
          'Based on the successful examples, what specific approaches could ', agent_name, ' have used? ',
          'Focus on concrete, actionable suggestions like specific phrases, procedures, or techniques. ',
          'List as short bullet points. If no improvements needed, respond with "Current approach was appropriate."'
        ),
        connection_id => 'us.vertex-ai-connection',
        endpoint => 'gemini-2.5-flash'
      ).result AS call_improvements
      
    FROM base
  ),

  -- Aggregate weekly performance by agent - this should preserve ALL agents
  weekly_agent_summary AS (
    SELECT 
      agent_id,
      agent_name,
      
      -- Call volume and performance metrics
      COUNT(*) as total_calls,
      ROUND(AVG(CAST(current_rating AS FLOAT64)), 1) as avg_rating,
      COUNT(CASE WHEN CAST(current_success AS STRING) = 'true' THEN 1 END) as successful_calls,
      ROUND(COUNT(CASE WHEN CAST(current_success AS STRING) = 'true' THEN 1 END) * 100.0 / COUNT(*), 1) as success_rate_pct,
      
      -- Aggregate all positives from individual calls
      STRING_AGG(
        CASE WHEN call_positives != 'No notable positives identified.' 
          THEN call_positives ELSE NULL END, 
        '\n\n' 
      ) as all_call_positives,
      
      -- Aggregate all negatives from individual calls  
      STRING_AGG(
        CASE WHEN call_negatives != 'No notable improvement areas identified.' 
          THEN call_negatives ELSE NULL END,
        '\n\n'
      ) as all_call_negatives,
      
      -- Aggregate all improvement suggestions
      STRING_AGG(
        CASE WHEN call_improvements != 'Current approach was appropriate.'
          THEN call_improvements ELSE NULL END,
        '\n\n'  
      ) as all_call_improvements
      
    FROM individual_call_analysis
    GROUP BY agent_id, agent_name
  )

  -- Final consolidated weekly report
  SELECT 
    agent_id,
    agent_name,
    total_calls,
    avg_rating,
    successful_calls,
    success_rate_pct,
    
    -- Consolidated positive themes
    AI.GENERATE(
      CONCAT(
        'Analyze all these positive agent behaviors from the week and create a consolidated summary. ',
        'Group similar behaviors and create short bullet points of key strengths:\n\n',
        'Agent: ', agent_name, '\n',
        'Total Calls: ', total_calls, '\n',
        'Avg Rating: ', avg_rating, '/10\n', 
        'Success Rate: ', success_rate_pct, '%\n\n',
        'All Positive Behaviors This Week:\n',
        COALESCE(all_call_positives, 'No notable positives identified across calls'), '\n\n',
        'Create 3-5 consolidated bullet points highlighting this agents key strengths and consistent positive behaviors.'
      ),
      connection_id => 'us.vertex-ai-connection',
      endpoint => 'gemini-2.5-flash'
    ).result AS weekly_positives,
    
    -- Consolidated improvement areas
    AI.GENERATE(
      CONCAT(
        'Analyze all improvement areas from this agents calls this week and create consolidated feedback. ',
        'Group similar issues and create short bullet points of key improvement opportunities:\n\n',
        'Agent: ', agent_name, '\n',
        'Total Calls: ', total_calls, '\n',
        'Avg Rating: ', avg_rating, '/10\n',
        'Success Rate: ', success_rate_pct, '%\n\n', 
        'All Improvement Areas This Week:\n',
        COALESCE(all_call_negatives, 'No notable improvement areas identified across calls'), '\n\n',
        'Create 3-5 consolidated bullet points highlighting consistent improvement opportunities. ',
        'Focus on patterns, not individual incidents.'
      ),
      connection_id => 'us.vertex-ai-connection', 
      endpoint => 'gemini-2.5-flash'
    ).result AS weekly_negatives,
    
    -- Best practices and coaching recommendations
    AI.GENERATE(
      CONCAT(
        'Based on this agents performance and successful examples from other agents, create specific coaching recommendations:\n\n',
        'Agent: ', agent_name, '\n',
        'Performance Summary:\n',
        '- Total Calls: ', total_calls, '\n',
        '- Avg Rating: ', avg_rating, '/10\n',
        '- Success Rate: ', success_rate_pct, '%\n\n',
        
        'Improvement Suggestions From Successful Examples:\n',
        COALESCE(all_call_improvements, 'No specific improvements identified'), '\n\n',
        
        'Provide:\n',
        '1. Top 3-4 specific coaching points with concrete actions\n',
        '2. Recommended phrases or approaches for common scenarios\n',
        '3. Skills to focus on for professional development\n',
        '4. Specific situations where agent should apply different techniques\n\n',
        'Make recommendations actionable and supportive, focusing on growth opportunities.'
      ),
      connection_id => 'us.vertex-ai-connection',
      endpoint => 'gemini-2.5-flash' 
    ).result AS coaching_recommendations,
    
    -- Evaluation metadata
    eval_week_start,
    eval_week_end,
    CURRENT_TIMESTAMP() AS report_generated_at
    
  FROM weekly_agent_summary
  ORDER BY total_calls DESC
);

Executing query with job ID: 765fdac1-a1cc-474e-991e-d404fa191c19
Query executing: 10.67s

In [ ]:
%%bigquery agent_eval

select * from `trilink-472019.database.weekly_agent_performance`;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
print(f"Agent Training Data Shape for 10 Agents :{agent_eval.shape[0]}")
agent_eval

Agent Training Data Shape for 10 Agents :10


,agent_id,agent_name,total_calls,avg_rating,successful_calls,success_rate_pct,weekly_positives,weekly_negatives,coaching_recommendations,eval_week_start,eval_week_end,report_generated_at
0,agent_006,Michael Brown,15,5.9,12,80.0,Here are Michael Brown's key strengths and con...,Here are the consolidated feedback and key imp...,"Michael Brown's performance, particularly his ...",2025-09-14,2025-09-20,2025-09-15 02:05:41.424954+00:00
1,agent_002,David Chen,12,6.0,10,83.3,David Chen consistently exhibits several key s...,Here are the consolidated improvement opportun...,David Chen shows a solid ability to resolve im...,2025-09-14,2025-09-20,2025-09-15 02:05:41.424954+00:00
2,agent_003,Maria Rodriguez,12,5.4,7,58.3,Maria Rodriguez consistently demonstrates a hi...,"Here's consolidated feedback for Maria, groupi...","Maria, your dedication to resolving customer i...",2025-09-14,2025-09-20,2025-09-15 02:05:41.424954+00:00
3,agent_010,Carlos Martinez,11,6.3,10,90.9,Carlos Martinez consistently demonstrates exce...,Here are the consolidated key improvement oppo...,"Carlos, your 90.9% success rate in resolving i...",2025-09-14,2025-09-20,2025-09-15 02:05:41.424954+00:00
4,agent_007,Jennifer Davis,8,5.4,6,75.0,Here are Jennifer Davis's key strengths and co...,Here's consolidated feedback for Jennifer Davi...,Here are specific coaching recommendations for...,2025-09-14,2025-09-20,2025-09-15 02:05:41.424954+00:00
5,agent_009,Ashley Johnson,7,6.3,4,57.1,Ashley Johnson consistently demonstrates a hig...,"Based on Ashley Johnson's calls this week, her...","Ashley, your success rate of 57.1% and average...",2025-09-14,2025-09-20,2025-09-15 02:05:41.424954+00:00
6,agent_005,Lisa Wang,7,5.1,4,57.1,Here are Lisa Wang's consolidated key strength...,Here are the consolidated feedback areas for L...,Lisa Wang's performance summary (Avg Rating: 5...,2025-09-14,2025-09-20,2025-09-15 02:05:41.424954+00:00
7,agent_004,James Thompson,7,6.1,4,57.1,Here is a consolidated summary of James Thomps...,Based on James Thompson's call performance thi...,Here are specific coaching recommendations for...,2025-09-14,2025-09-20,2025-09-15 02:05:41.424954+00:00
8,agent_008,Robert Kim,3,6.0,3,100.0,Here are Robert Kim's key strengths and consis...,"Here is consolidated feedback for Robert Kim, ...","Robert, your 100% success rate in resolving th...",2025-09-14,2025-09-20,2025-09-15 02:05:41.424954+00:00
9,agent_001,Sarah Mitchell,2,4.5,0,0.0,Here are 3-5 consolidated bullet points highli...,Here is a consolidated feedback for Sarah Mitc...,Here are specific coaching recommendations for...,2025-09-14,2025-09-20,2025-09-15 02:05:41.424954+00:00


In [ ]:
def display_agent_performance(df_agents, i=None):
    """
    Display agent performance evaluation including ratings, feedback, and coaching recommendations.
    
    Parameters:
    df_agents (DataFrame): DataFrame containing agent performance data
    i (int, optional): Index of the agent to display. If None, selects a random agent.
    """
    
    if i is None:
        # Select a random agent
        i = np.random.randint(0, df_agents.shape[0])
    
    # Ensure index is within bounds
    if i < 0 or i >= df_agents.shape[0]:
        print(f"Error: Index {i} is out of bounds. DataFrame has {df_agents.shape[0]} rows.")
        return
    
    # Get agent data
    agent_data = df_agents.iloc[i]
    
    print(f"Agent: {agent_data['agent_name']} (ID: {agent_data['agent_id']})")
    print(f"Evaluation Period: {agent_data['eval_week_start']} to {agent_data['eval_week_end']}")
    print(f"Average Rating: {agent_data['avg_rating']:.2f}")
    print(f"Successful Calls: {agent_data['successful_calls']} | Success Rate: {agent_data['success_rate_pct']:.1f}%")
    print("=" * 80)
    
    print("THINGS YOU EXCEL AT:")
    print("-" * 20)
    print(agent_data['weekly_positives'])
    print()
    
    print("THINGS YOU COULD IMPROVE ON:")
    print("-" * 30)
    print(agent_data['weekly_negatives'])
    print()
    
    print("HERE ARE SHORT RECOMMENDATIONS FOR IMPROVING AND GETTING THE BEST:")
    print("-" * 70)
    print(agent_data['coaching_recommendations'])
    print("=" * 80)

In [ ]:
display_agent_performance(df_agents=agent_eval.sort_values(by='avg_rating',ascending=True),i=0)

Agent: Sarah Mitchell (ID: agent_001)
Evaluation Period: 2025-09-14 to 2025-09-20
Average Rating: 4.50
Successful Calls: 0 | Success Rate: 0.0%
THINGS YOU EXCEL AT:
--------------------
Here are 3-5 consolidated bullet points highlighting Sarah Mitchell's key strengths and consistent positive behaviors:

*   **Exceptional Customer Engagement & Empathy:** Consistently demonstrated active listening, validated customer frustrations with genuine understanding, and maintained a professional, respectful, and apologetic tone throughout complex interactions, from greeting to closing.
*   **Proactive & Resourceful Problem-Solving:** Showed strong initiative by offering a comprehensive range of solutions (e.g., diagnostics, specialized tickets, tech visits, alternative plans, escalations), clearly explaining the *why* and *how* for each to address multifaceted issues.
*   **Clear Communication & Expectation Management:** Excelled at communicating findings, explaining discrepancies, articulating 

#### Action 3 - ML-Powered Personalized Retention Emails

**Idea**: Transform traditional churn prediction models into hyper-personalized retention campaigns that address individual customer risk factors

**The Problem**:
- Business ML models predict churn events but only generate generic target lists
- All high-risk customers receive identical "cookie-cutter" retention messages
- Rich model insights about WHY customers churn are completely ignored
- A/B testing still uses generic content variations instead of personalized approaches

**The Solution**:
- **Build XGBoost churn models** in BigQuery ML for internet and mobile customers  
- **Use ML.EXPLAIN_PREDICT** to extract individual feature contributions (not just probability)
- **AI-analyze risk factors** to convert technical features into customer pain points
- **Vector-match business solutions** to find proven approaches for similar problems
- **Generate personalized emails** that address specific churn drivers for each customer

**Process Flow**:
- **Target Segmentation**: Filter high-risk customers (95%+ churn probability) from specific demographics (young employed, work-from-home, upper-middle income)
- **Feature Attribution Analysis**: Extract why each customer is at risk using ML.EXPLAIN_PREDICT feature contributions  
- **Problem Description Generation**: AI converts technical risk factors into comprehensive customer problem statements
- **Business Solution Matching**: Vector search against solution database to find proven retention approaches for similar issues
- **Service Portfolio Analysis**: Gather current customer services (mobile, internet, security) to identify cross-sell opportunities
- **Personalized Email Creation**: Generate retention emails addressing specific risk factors with relevant service recommendations

**Why This Works**:
- **Addresses root causes** instead of generic retention offers
- **Speaks to actual pain points** each customer experiences  
- **Provides relevant solutions** based on their specific risk factors
- **Maximizes ML model intelligence** beyond simple targeting
- **Increases retention effectiveness** through precise personalization

In [ ]:
%%bigquery

select count(*),data_overage_frequency from `trilink-472019.database.mobile_churn_data`
  WHERE tenure_years > 1 
         AND age < 30
         AND age>25 
         AND mobile_churn = 0 
         AND work_from_home_flag = True 
         AND income_bracket = 'Upper_Middle'
group by data_overage_frequency
order by data_overage_frequency
;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_,data_overage_frequency
0,186,0
1,2,5
2,6,7
3,3,8
4,5,9
5,2,10
6,5,11
7,1,12
8,2,13
9,2,14


- Let's try to create targeted retention email for you customers who are working from home and have tenure more than 1 year etc. 
- A table with personalized retention email will be created for those customers with very high churn probability with the best actions possible to reduce their churn
- Business can simply export/connect this table to the marketing tool and send it on demand to customers
- Any segments can be targetd here. Demo with this subset of ~200 customers since it's running on my personal Gemini account
- Takes little as 5 mins for 1000 personalized emails in simple bigquery system below at a cost of less than ~2$ (Using Gemini 2.5 Flash - could be even cheaper as models scales or using smaller mdoel)
- Demo shown below is for mobile customers but the same can be replicated to the internet customers customized prompt to internet services. We have the churn model created for internet services as well

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE `trilink-472019.database.mobile_retention_email_young_employed` AS (
  
-- Base customer churn predictions for multiple customers
WITH customer_churn_base AS (
  SELECT
    customer_id,
    predicted_mobile_churn,
    ROUND(probability * 100, 1) AS churn_probability_percent,
    age,
    plan_type,
    monthly_cost,
    contract_type,
    tenure_years,
    data_overage_frequency,
    family_plan_flag,
    household_income,
    family_size,
    home_ownership,
    work_from_home_flag,
    education_level,
    life_stage,
    income_per_person,
    cost_per_line,
    fiber_availability,
    neighborhood_crime_rate,
  
    
    -- Risk category
    CASE
      WHEN probability >= 0.8 THEN 'CRITICAL RISK'
      WHEN probability >= 0.6 THEN 'HIGH RISK'
      WHEN probability >= 0.4 THEN 'MEDIUM RISK'
      ELSE 'LOW RISK'
    END AS risk_category,
    
    top_feature_attributions
  FROM (
    SELECT *
    FROM ML.EXPLAIN_PREDICT(
      MODEL `trilink-472019.database.mobile_churn_predictor`,
      (SELECT * FROM `trilink-472019.database.mobile_churn_data`
       WHERE tenure_years > 1 
         AND age < 30
         AND age>25 
         AND mobile_churn = 0 
         AND work_from_home_flag = True 
         AND income_bracket = 'Upper_Middle'
      )
    )
    WHERE probability > 0.95
  )
) ,


-- Security services aggregation
customer_security AS (
  SELECT 
    customer_id,
    STRING_AGG(device_type, ', ' ORDER BY device_type) AS device_types_concat,
    COUNT(device_type) AS total_security_devices
  FROM `trilink-472019.database.security_df`
  WHERE customer_id IN (SELECT customer_id FROM customer_churn_base)
  GROUP BY customer_id
),

-- Active internet services
customer_internet AS (
  SELECT 
    customer_id,
    speed_mbps AS internet_speed,
    plan_tier AS internet_plan,
    contract_type AS internet_contract_type,
    internet_tenure_days
  FROM `trilink-472019.database.internet_df`
  WHERE customer_id IN (SELECT customer_id FROM customer_churn_base)
    AND internet_churn = 0
),

-- Active mobile services (non-churned)
customer_mobile AS (
  SELECT 
    customer_id,
    plan_type AS current_mobile_plan,
    contract_type AS current_mobile_contract_type,
    data_overage_frequency AS current_mobile_data_overage,
    monthly_cost AS current_mobile_cost
  FROM `trilink-472019.database.mobile_df`
  WHERE customer_id IN (SELECT customer_id FROM customer_churn_base)
    AND mobile_churn = 0
),

-- Problem descriptions for business solution matching (AI-generated)
customer_problems AS (
  SELECT 
    cb.customer_id,
    AI.GENERATE(
      CONCAT(
        'Analyze this mobile customer situation and identify their key problems/pain points in under 150 words. Start with "Customer is facing...":\\n\\n',
        'Customer Profile:\\n',
        '- Age: ', CAST(cb.age AS STRING), ' years (', cb.life_stage, ')\\n',
        '- Income: $', CAST(cb.household_income AS STRING), ' household, $', CAST(CAST(cb.income_per_person AS INT64) AS STRING), ' per person\\n',
        '- Family: ', CAST(cb.family_size AS STRING), ' people, ', cb.home_ownership, '\\n',
        '- Education: ', cb.education_level, '\\n',
        '- Works from home: ', CAST(cb.work_from_home_flag AS STRING), '\\n',
        'Mobile Service:\\n',
        '- Plan: ', cb.plan_type, '\\n',
        '- Cost: $', CAST(cb.monthly_cost AS STRING), '/month ($', CAST(CAST(cb.cost_per_line AS INT64) AS STRING), ' per line)\\n',
        '- Contract: ', cb.contract_type, '\\n',
        '- Tenure: ', CAST(cb.tenure_years AS STRING), ' years\\n',
        '- Data overages: ', CAST(cb.data_overage_frequency AS STRING), ' per month\\n',
        '- Family plan: ', CAST(cb.family_plan_flag AS STRING), '\\n',
        '- Fiber available: ', CAST(cb.fiber_availability AS STRING), '\\n\\n',
        'Churn Analysis:\\n',
        '- Risk: ', CAST(cb.churn_probability_percent AS STRING), '% (', cb.risk_category, ')\\n',
        '- Key factors: ', ARRAY_TO_STRING(
          ARRAY(
            SELECT CONCAT(feature, ': ', CAST(ROUND(attribution, 3) AS STRING))
            FROM UNNEST(cb.top_feature_attributions)
            ORDER BY ABS(attribution) DESC
            LIMIT 5
          ), ', '
        ), '\\n\\n',
        'Task: Write a detailed problem analysis identifying:\\n',
        'Why they are at risk of churning? What could be the possible reasons and or frustration customer is facing\\n',
        'Format as a comprehensive problem statement for vector similarity matching with business solutions later.'
      ),
      connection_id => 'us.vertex-ai-connection',
      endpoint => 'gemini-2.5-flash'
    ).result AS problem_description
  FROM customer_churn_base cb
),

-- Generate embeddings for customer problems
customer_problem_embeddings AS (
  SELECT *
  FROM ML.GENERATE_EMBEDDING(
    MODEL `trilink-472019.database.text_embedding_005_model`,
    (SELECT 
       problem_description AS content,
       customer_id
     FROM customer_problems),
    STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_DOCUMENT' AS task_type)
  )
),

-- Business solution matching
business_solutions AS (
  SELECT 
    vs.query.customer_id,
    vs.base.problem AS similar_matched_problem,
    vs.base.solution AS recommended_solution,
    vs.base.sector AS sector,
    vs.distance AS solution_match_score
  FROM VECTOR_SEARCH(
    TABLE `trilink-472019.database.business_solution_embeddings`,
    'problem_embedding',
    TABLE customer_problem_embeddings,
    'ml_generate_embedding_result',
    top_k => 1,
    distance_type => 'COSINE'
  ) AS vs
),

-- Consolidated customer data
customer_consolidated AS (
  SELECT 
    cb.*,
    
    -- Security services
    COALESCE(cs.device_types_concat, 'None') AS device_types_concat,
    COALESCE(cs.total_security_devices, 0) AS total_security_devices,
    CASE WHEN cs.customer_id IS NOT NULL THEN 1 ELSE 0 END AS security_customer,
    
    -- Internet services
    ci.internet_speed,
    ci.internet_plan,
    ci.internet_contract_type,
    ci.internet_tenure_days,
    CASE 
      WHEN ci.customer_id IS NULL THEN 'No internet service'
      ELSE 'Active internet customer' 
    END AS internet_customer_status,
    
    -- Mobile services (from churn table vs active table)
    cm.current_mobile_plan,
    cm.current_mobile_contract_type,
    cm.current_mobile_data_overage,
    cm.current_mobile_cost,
    CASE 
      WHEN cm.customer_id IS NULL THEN 'Mobile service at risk'
      ELSE 'Active mobile customer'
    END AS mobile_customer_status,
    
    -- Business solutions
    bs.similar_matched_problem,
    bs.recommended_solution,
    bs.sector,
    bs.solution_match_score
    
  FROM customer_churn_base cb
  LEFT JOIN customer_security cs ON cb.customer_id = cs.customer_id
  LEFT JOIN customer_internet ci ON cb.customer_id = ci.customer_id
  LEFT JOIN customer_mobile cm ON cb.customer_id = cm.customer_id
  LEFT JOIN business_solutions bs ON cb.customer_id = bs.customer_id
)

-- Final output with email generation
SELECT
  cc.customer_id,
  churn_probability_percent,
  risk_category,
  plan_type,
  monthly_cost,
  internet_customer_status,
  mobile_customer_status,
  security_customer,
  
  -- Use the AI-generated problem description from solution matching
  cp.problem_description AS problem_analysis,
  similar_matched_problem,
  recommended_solution,
  solution_match_score,

  -- Enhanced retention email generation
  AI.GENERATE(
    CONCAT(
      'Create a personalized retention email for this at-risk mobile customer. Address their specific churn risks and suggest relevant additional services:\\n\\n',
      
      '=== CUSTOMER OVERVIEW ===\\n',
      'Customer ID: ', cc.customer_id, '\\n',
      'Churn Risk: ', CAST(churn_probability_percent AS STRING), '% (', risk_category, ')\\n',
      'Profile: ', CAST(age AS STRING), '-year-old ', life_stage, '\\n',
      'Family: ', CAST(family_size AS STRING), ' people, ', home_ownership, '\\n',
      'Income: $', CAST(household_income AS STRING), ' household\\n',
      'Work Setup: ', CASE WHEN work_from_home_flag THEN 'Works from home' ELSE 'Traditional workplace' END, '\\n\\n',
      
      '=== CURRENT SERVICE PORTFOLIO ===\\n',
      'Mobile Service: ', CAST(tenure_years AS STRING), ' years with us\\n',
      '- Plan: ', plan_type, ' ($', CAST(monthly_cost AS STRING), '/month)\\n',
      '- Contract: ', contract_type, '\\n',
      '- Data Issues: ', CAST(data_overage_frequency AS STRING), ' overages/month\\n',
      '- Family Plan: ', CAST(family_plan_flag AS STRING), '\\n\\n',
      
      'Internet Service: ', internet_customer_status, '\\n',
      CASE WHEN internet_customer_status = 'Active internet customer' THEN
        CONCAT('- Plan: ', internet_plan, ' (', CAST(internet_speed AS STRING), ' Mbps)\\n',
               '- Contract: ', internet_contract_type, '\\n')
        ELSE CONCAT('- Opportunity: Fiber available = ', CAST(fiber_availability AS STRING), '\\n') END,
      
      'Security Service: ', CASE WHEN security_customer = 1 
        THEN CONCAT('Active (', CAST(total_security_devices AS STRING), ' devices: ', device_types_concat, ')')
        ELSE 'Not subscribed' END, '\\n',
        CASE WHEN security_customer = 0 THEN 
        CONCAT('- Area Crime Rate: ', neighborhood_crime_rate, '\\n') ELSE '' END, '\\n',

      
      '=== CHURN RISK FACTORS ===\\n',
      'Risk Level: ', risk_category, ' (', CAST(churn_probability_percent AS STRING), '%)\\n',
      'Key Risk Drivers:\\n',
      ARRAY_TO_STRING(
        ARRAY(
          SELECT CONCAT('• ', feature, ': ', 
            CASE WHEN attribution > 0 THEN 'Increasing churn risk' 
                 ELSE 'Reducing churn risk' END,
            ' (', CAST(ROUND(attribution, 3) AS STRING), ')')
          FROM UNNEST(top_feature_attributions)
          ORDER BY ABS(attribution) DESC
          LIMIT 5
        ), '\\n'
      ), '\\n\\n',
      
      '=== BUSINESS SOLUTION GUIDANCE ===\\n',
      'From the business solution book, we found a problem most similar to this customers issue: ', COALESCE(similar_matched_problem, 'Standard mobile service retention'), '\\n',
      'And the business solution to this was: ', COALESCE(recommended_solution, 'Personalized value demonstration and service optimization'), '\\n',
      
      '=== AVAILABLE SERVICE OPTIONS ===\\n',
      'MOBILE PLANS:\\n',
      '• Limited 2GB: $30/month\\n',
      '• Limited 5GB: $40/month\\n', 
      '• Limited 10GB: $50/month\\n',
      '• Unlimited Standard: $70/month\\n',
      '• Unlimited Premium: $90/month (includes Disney+)\\n\\n',
      
      'INTERNET PLANS:\\n',
      '• Basic: 25 Mbps at $40/month\\n',
      '• Standard: 100 Mbps at $70/month\\n',
      '• Premium Gig: 1000 Mbps at $100/month\\n\\n',
      
      'SECURITY OPTIONS:\\n',
      '• Starter Kit: Motion + Window sensors ($60)\\n',
      '• Smart Security: Doorbell + Indoor Camera ($200)\\n',
      '• Complete Protection: Full system with monitoring ($350 + $30/month)\\n\\n',
      
      '=== EMAIL REQUIREMENTS ===\\n',
      '**PRIMARY GOAL**: Prevent churn by addressing specific risk factors and demonstrating value. Prvoide the best solutions customer could do or we as business could do to solve the issues for customer\\n',
      '**SECONDARY GOAL**: Suggest relevant services (cros-sell) they dont have as long as its logical and is not a deterrance to customer retention efforts\\n\\n',
      
      '**STRUCTURE**:\\n',
      '1. Warm personal greeting (no placeholder names)\\n',
      '2. Acknowledge their ', CAST(tenure_years AS STRING), '-year relationship\\n',
      '3. Address top 2-3 churn risk factors specifically\\n',
      '4. Apply the business solution approach provided\\n',
      '5. Present 1-2 relevant additional services (only if they dont have them)\\n',
      '6. Offer meaningful retention incentive '
            '- YOU ARE ALLOWED TO OFFER ONLY A Maximum 20$ discount. YOU CANNOT OFFER ANY RETENTION OFFER BY DECREASING RATES MORE THAN BY 20$\\n',
            '- RETENTION OFFERS ARE TO INCREASE THE CUSTOMER LIFETIME VALUE. OFFERS PROVIDED SHOULD ALWAYS CONSIDER AND OPTIMIZE FOR THAT',
            '- YOU CAN OFFER FREE FIRST MONTH OFFER FOR MOBILE OR INTERNET CROSS SELL OFFERS IF YOU MAKE ONE\\n'
            '- YOU CAN MAKE 50$ OFFER FOR SECURITY DEVICES IF YOU MAKE ANY CROSS-SELL FOR FOR IT ONE\\n'
      '7. Clear next steps and contact info\\n\\n',
      
      '**CROSS-SELL LOGIC**:\\n',
      '• No Internet + WFH = Emphasize internet bundle\\n',
      '• No Security + High Crime = Home protection focus\\n',
      '• Data Overages = Unlimited plan upgrade\\n',
      '• Family + Limited Plan = Family plan benefits\\n',
      '• Keep suggestions brief and value-focused\\n',
      '• Include small incentives for new services\\n\\n',
      
      '**TONE**: Empathetic, solution-focused, value-driven, personally crafted\\n',
      '**LENGTH**: 350-450 words\\n',
      '**SIGNATURE**: TriLink Customer Success Team\\n',
      '**CONTACT**: 1-800-TRILINK or customercare@trilink.com\\n\\n',
      
      'Generate the complete retention email:'
    ),
    connection_id => 'us.vertex-ai-connection',
    endpoint => 'gemini-2.5-flash'
  ).result AS personalized_retention_email,
  
  -- Additional columns for analysis
  top_feature_attributions,
  device_types_concat,
  total_security_devices,
  internet_speed,
  internet_plan

FROM customer_consolidated cc
LEFT JOIN customer_problems cp ON cc.customer_id = cp.customer_id
ORDER BY churn_probability_percent DESC
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery  mobile_email_campaign_retention

select * from `trilink-472019.database.mobile_retention_email_young_employed`;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
mobile_email_campaign_retention.head(5)

,customer_id,churn_probability_percent,risk_category,plan_type,monthly_cost,internet_customer_status,mobile_customer_status,security_customer,problem_analysis,similar_matched_problem,recommended_solution,solution_match_score,personalized_retention_email,top_feature_attributions,device_types_concat,total_security_devices,internet_speed,internet_plan
0,C00096527,96.5,CRITICAL RISK,Unlimited_Premium,215,Active internet customer,Active mobile customer,0,Customer is facing critical churn risk primari...,Customer with Limited_2GB plan hitting data li...,Upgrade to Unlimited_Standard $69/month elimin...,0.235632,Subject: An Important Update for Your TriLink ...,"[{'feature': 'tenure_years', 'attribution': 0....",None,0,25,Basic_25
1,C00009009,96.5,CRITICAL RISK,Unlimited_Standard,174,No internet service,Active mobile customer,1,Customer is facing a critical 96.5% churn risk...,Customer with Limited_2GB plan hitting data li...,Upgrade to Unlimited_Standard $69/month elimin...,0.263261,**Subject: An Important Update on Your TriLink...,"[{'feature': 'tenure_years', 'attribution': 0....","Indoor_Camera, Motion_Sensor, Smart_Doorbell, ...",4,<NA>,None
2,C00090448,96.5,CRITICAL RISK,Unlimited_Standard,222,No internet service,Active mobile customer,1,Customer is facing perceived poor value for mo...,Customer with Limited_2GB plan hitting data li...,Upgrade to Unlimited_Standard $69/month elimin...,0.278812,Subject: An Important Message for Valued TriLi...,"[{'feature': 'tenure_years', 'attribution': 0....","Motion_Sensor, Outdoor_Camera, Security_Panel,...",4,<NA>,None
3,C00079038,96.5,CRITICAL RISK,Unlimited_Standard,227,Active internet customer,Active mobile customer,1,Customer is facing critical churn risk primari...,Customer with Limited_2GB plan hitting data li...,Upgrade to Unlimited_Standard $69/month elimin...,0.259813,Subject: An Important Review of Your TriLink S...,"[{'feature': 'tenure_years', 'attribution': 0....","Indoor_Camera, Outdoor_Camera, Security_Panel,...",5,100,Standard_100
4,C00067487,96.5,CRITICAL RISK,Unlimited_Premium,222,Active internet customer,Active mobile customer,1,Customer is facing critical churn risk (96.5%)...,Customer with Limited_2GB plan hitting data li...,Upgrade to Unlimited_Standard $69/month elimin...,0.280363,Subject: An Important Update from TriLink Rega...,"[{'feature': 'tenure_years', 'attribution': 0....","Indoor_Camera, Smart_Doorbell, Smart_Lock, Win...",4,1000,Premium_Gig


In [ ]:
def display_customer_retention(df_customers, i=None):
    """
    Display customer retention analysis including problem matching, solutions, and personalized retention emails.
    
    Parameters:
    df_customers (DataFrame): DataFrame containing customer retention data
    i (int, optional): Index of the customer to display. If None, selects a random customer.
    """
    
    # Display total number of customers in the dataset
    print(f"Total Customers in Dataset: {df_customers.shape[0]}")
    print("=" * 80)
    print()
    
    if i is None:
        # Select a random customer
        i = np.random.randint(0, df_customers.shape[0])
    
    # Ensure index is within bounds
    if i < 0 or i >= df_customers.shape[0]:
        print(f"Error: Index {i} is out of bounds. DataFrame has {df_customers.shape[0]} rows.")
        return
    
    # Get customer data
    customer_data = df_customers.iloc[i]
    
    print(f"Customer: {customer_data['customer_id']}")
    print(f"Churn Risk: {customer_data['risk_category']} ({customer_data['churn_probability_percent']}%)")
    print(f"Internet Plan: {customer_data['internet_plan']} | Speed: {customer_data['internet_speed']}")
    print(f"Devices: {customer_data['device_types_concat']} | Security Devices: {customer_data['total_security_devices']}")
    print(f"Solution Match Score: {customer_data['solution_match_score']:.2f}")
    print("=" * 80)
    
    print("PROBLEM ANALYSIS:")
    print("-" * 20)
    pprint(customer_data['problem_analysis'])
    print()
    
    print("PERSONALIZED RETENTION EMAIL:")
    print("-" * 35)
    pprint(customer_data['personalized_retention_email'])
    print()
    
    print("TOP FEATURE ATTRIBUTIONS:")
    print("-" * 30)
    pprint(customer_data['top_feature_attributions'])
    print("=" * 80)

In [ ]:
display_customer_retention(df_customers=mobile_email_campaign_retention)

Total Customers in Dataset: 22

Customer: C00009465
Churn Risk: CRITICAL RISK (96.3%)
Internet Plan: Premium_Gig | Speed: 1000
Devices: Indoor_Camera, Outdoor_Camera, Security_Panel, Smart_Doorbell, Smart_Lock, Window_Sensor | Security Devices: 6
Solution Match Score: 0.28
PROBLEM ANALYSIS:
--------------------
('Customer is facing critical churn risk primarily driven by a perception of '
 'poor value for money, reflected in their high monthly mobile cost of $176 '
 'for four lines on an Unlimited_Standard plan. This young family, with a WFH '
 'parent, exhibits cost sensitivity despite a solid income, likely seeking '
 'greater financial flexibility or better features from competitors. Their '
 "zero data overages suggest potential underutilization of the 'unlimited' "
 'benefits, while being tied to a 12-month contract adds to dissatisfaction. '
 'This combination makes them highly susceptible to alternative, more '
 'flexible, or lower-cost offers, especially with readily available 

In [ ]:
display_customer_retention(df_customers=mobile_email_campaign_retention)

Total Customers in Dataset: 22

Customer: C00090448
Churn Risk: CRITICAL RISK (96.5%)
Internet Plan: None | Speed: <NA>
Devices: Motion_Sensor, Outdoor_Camera, Security_Panel, Smart_Doorbell | Security Devices: 4
Solution Match Score: 0.28
PROBLEM ANALYSIS:
--------------------
('Customer is facing perceived poor value for money from their high-cost '
 "'Unlimited_Standard' plan ($222/month) after nearly three years of loyalty, "
 'likely exacerbated by a lack of recognition or better new-customer offers. '
 'Their work-from-home status makes them highly dependent on consistent, '
 "reliable data speeds, which a 'Standard' plan may not adequately provide, "
 'leading to potential frustration with service quality and reliability '
 'critical for their livelihood. This combination of high cost, long tenure '
 'without perceived benefits, and potential service inadequacy for their WFH '
 'needs drives their critical churn risk, facilitated by a month-to-month '
 'contract.')

PERSONALIZED

#### Action 4 - Automated Ticket Handling for Home Security

- Idea: When customers have issues with the devices, they usually call/add and create a ticket for resolution. Often times what happens is the customer explaining what the problem is and the agent/bot creates a ticket for them adding some details of what customer told their issue is. Then a technician is assigned that ticket and visits the customers site to solve it in few days as fast as possible. However there are few problems in this traditional approach:-
    - Information Loss: The agent who handles customers issue won't be highly technical as is most customers. So what's noted down as issue or identified from the software doesn't explain the actual issues that might be there for a technical person. So the issue noted down might not be covering the whole base and technicain when they arrive at site could get stumped with different issues for which they might not have been prepared for
    - Severity of issue: Oftentimes a good number of tickets created are DIY tasks. FOr example if a customer finds their doorbell camera not having the 2 way audi working, it could be a software issue that could be solved with a software update. You don't need to send a technician for that. This is a huge waste of resource, money and the technicians valued time that could be focused more technical issue. No point in techncian driving in heavy traffic to do a app reboot which customer can do on their own, as long we provide them with step by step instructions
    - Technician preparedness: Technciains gets assigned with tickets with tasks assigned with target number of tickets. But it would be more practial and idea to assign the tickets effectively with their speciality and of severity instead of quantity

    So the ticket creation now will also ask the customer to upload images of the issue/device issue. Now we will use these images as well as the issue customer described with AI.GENERATE_TABLE and OBJ.MAKE_REF to automatically understand what device the issue is for, is it a hardware or software issue, does it need a on site visit by a technician, if Yes a report for technician to understand what the issue is detailed from image and issue description, if no technician needed, a step by step DIY instruction to send to customer as email or on a guided call to solve issue.

    This can be ran as a daily job on all the issues logged for that day. This is going to save a lot of time and money. Also with this structured issue data created, there can be much more efficient technician assignment where they are assigned by severity, city, device type, location etc  

In [35]:
from IPython.display import display, Image, HTML
import requests
from io import BytesIO
from google.cloud import storage

In [4]:
%%bigquery

--create a gemini 2.5 flash model connection to use in our AI.GENERAT_TABLE call

CREATE OR REPLACE MODEL `trilink-472019.database.gemini_flash`
REMOTE WITH CONNECTION `us.vertex-ai-connection`
  OPTIONS(ENDPOINT = 'gemini-2.5-flash');

Query is running:   0%|          |

""


- We have the ticket_id, customer_id, the issue description added in ticket by customer/agent, the URI of the image clicked by customer which shows what the issue was

In [5]:
%%bigquery
--Let's load the issues data we have for Trilink homesecurity. In the sample data created we have 100 tickets for which we will run the Multimodal process


select * from `trilink-472019.database.active_customer_tickets` limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

,ticket_id,customer_id,technician_id,issue_description,gcs_uri
0,TICKET_002,C00025032,tech001,The touchscreen on our Security Panel is compl...,gs://trilink/ticket-images/TICKET_002.jpeg
1,TICKET_007,C00054731,tech001,The time on my keypad is off by about an hour ...,gs://trilink/ticket-images/TICKET_007.jpeg
2,TICKET_010,C00008654,tech001,My security system is interfering with my gara...,gs://trilink/ticket-images/TICKET_010.jpeg
3,TICKET_013,C00071155,tech001,There are small black dots or specks all over ...,gs://trilink/ticket-images/TICKET_013.jpeg
4,TICKET_017,C00071465,tech001,The timestamp on my recorded video clips is wr...,gs://trilink/ticket-images/TICKET_017.jpeg
5,TICKET_021,C00097411,tech001,The little magnet piece for my window sensor f...,gs://trilink/ticket-images/TICKET_021.jpeg
6,TICKET_025,C00053965,tech001,The adhesive on my door sensor has failed and ...,gs://trilink/ticket-images/TICKET_025.jpeg
7,TICKET_029,C00030278,tech001,The plastic on my window sensor has yellowed b...,gs://trilink/ticket-images/TICKET_029.jpeg
8,TICKET_032,C00071732,tech001,My smart lock keypad is not lighting up when I...,gs://trilink/ticket-images/TICKET_032.jpeg
9,TICKET_036,C00098830,tech001,The motion detection on my doorbell is too sen...,gs://trilink/ticket-images/TICKET_036.jpeg


- We will now create a new table with the images added is as ObjectRef field which we can use in BigQuery to query on multimodal objects easily
- We will also bring in some customer details to get personalized content further down the line
- We can see the image_ref created which will be used further down to add curated contents

In [6]:
%%bigquery
CREATE OR REPLACE TABLE `trilink-472019.database.active_customer_tickets_mm` AS
SELECT
  a.*,
  OBJ.FETCH_METADATA(OBJ.MAKE_REF(gcs_uri, 'us.vertex-ai-connection')) as image_ref,
  b.age,b.city,b.life_stage,b.home_type,b.work_from_home_flag,b.family_size,b.home_ownership,b.postal_code
FROM `trilink-472019.database.active_customer_tickets` a
left join `trilink-472019.database.customer_df` b
on a.customer_id=b.customer_id;

select * from `trilink-472019.database.active_customer_tickets_mm` limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

,ticket_id,customer_id,technician_id,issue_description,gcs_uri,image_ref,age,city,life_stage,home_type,work_from_home_flag,family_size,home_ownership,postal_code
0,TICKET_051,C00044659,tech001,The smoke detector in my hallway went off for ...,gs://trilink/ticket-images/TICKET_051.jpeg,{'uri': 'gs://trilink/ticket-images/TICKET_051...,64,Hamilton,Empty_Nest,Townhouse,False,1,Own,L7S 9U3
1,TICKET_001,C00044659,tech002,My main Security Panel is beeping every minute...,gs://trilink/ticket-images/TICKET_001.jpeg,{'uri': 'gs://trilink/ticket-images/TICKET_001...,64,Hamilton,Empty_Nest,Townhouse,False,1,Own,L7S 9U3
2,TICKET_058,C00042917,tech002,I have a heat detector in my attic. It went of...,gs://trilink/ticket-images/TICKET_058.jpeg,{'uri': 'gs://trilink/ticket-images/TICKET_058...,54,Mississauga,Empty_Nest,Single_Family,False,1,Own,L8G 8H8
3,TICKET_008,C00042917,tech004,I need to reset my master code. My ex-partner ...,gs://trilink/ticket-images/TICKET_008.jpeg,{'uri': 'gs://trilink/ticket-images/TICKET_008...,54,Mississauga,Empty_Nest,Single_Family,False,1,Own,L8G 8H8
4,TICKET_005,C00078974,tech002,I can't arm my system. The panel says 'Device ...,gs://trilink/ticket-images/TICKET_005.jpeg,{'uri': 'gs://trilink/ticket-images/TICKET_005...,58,Toronto,Empty_Nest,Townhouse,True,1,Own,M6G 0L8
5,TICKET_055,C00078974,tech003,I'm renovating my kitchen and the drywall dust...,gs://trilink/ticket-images/TICKET_055.jpeg,{'uri': 'gs://trilink/ticket-images/TICKET_055...,58,Toronto,Empty_Nest,Townhouse,True,1,Own,M6G 0L8
6,TICKET_017,C00071465,tech001,The timestamp on my recorded video clips is wr...,gs://trilink/ticket-images/TICKET_017.jpeg,{'uri': 'gs://trilink/ticket-images/TICKET_017...,76,Toronto,Senior,Single_Family,False,1,Own,M6W 3D3
7,TICKET_011,C00004651,tech003,My outdoor camera over the garage has a big sp...,gs://trilink/ticket-images/TICKET_011.jpeg,{'uri': 'gs://trilink/ticket-images/TICKET_011...,30,Mississauga,Young_Family,Single_Family,True,2,Own,L6E 3Z4
8,TICKET_074,C00078739,tech005,The solar panel for my outdoor camera doesn't ...,gs://trilink/ticket-images/TICKET_074.jpeg,{'uri': 'gs://trilink/ticket-images/TICKET_074...,57,Mississauga,Empty_Nest,Single_Family,False,2,Own,L9M 6H2
9,TICKET_067,C00017724,tech002,My outdoor camera view is blocked by a large t...,gs://trilink/ticket-images/TICKET_067.jpeg,{'uri': 'gs://trilink/ticket-images/TICKET_067...,36,Toronto,Young_Family,Single_Family,True,2,Own,M2D 8N4


We will now create a table with the solutions for all tickets. The inputs passed to query are the image ObjectRef, issue descrption as well as some customer features for personalization.
Using AI.GENERATE_TABLE with  Gemini 2.5-Flash we will now create a table that will have:-

- device_name :  Idenitfy what security device the issue is primarily on
- harware_software: A string that tells if it's a gardware, software or neither problem. Software problems can be troubleshooted from HQ itself
- need_on_site_visit: A boolean flag weherwe ask the model from image added and issue desc whether a technician needs to be assigned for this. Most of the issues might be simpler where we can guide the customer through problems as DIY task
- urgency: A numeric rating from 1-5 showing 5 as most urgent issue. This helps business to priortize technician assignment better than the current random assignment 
- technician_report: When a technician is needed on site for solution, we will ask the model to create a structured formatted report for the technciain to understand the problem. Information loss can be corrected now that we will take models help to get the whole info from iamge and issue desc to technciain in a standard format with more details on tools needed, expected time it might take etc. This will help technicians have a heads up already on what to expect, what to do etc.
- diy_instructions: If a technicial on site was not needed, then we ask the model to create a personalized step by step instruction to be send to the customer via an email or a guided call, to solve their problem where customer can do it themselves the solution from instructions we pass as long as a solution is there. If the issue was hardware beyond repaair we will send them the message that a new one can be send right on to replace


In [31]:
%%bigquery

CREATE OR REPLACE TABLE `trilink-472019.database.active_customer_tickets_solutions` AS
(SELECT 
  ticket_id,
  issue_description,
  device_name,
  hardware_or_software,
  need_on_site_visit,
  urgency,
  technician_report,
  diy_instructions,
  gcs_uri
FROM AI.GENERATE_TABLE(
  MODEL `trilink-472019.database.gemini_flash`,
  (
    SELECT (
      'CUSTOMER PROFILE: ',
      'Age: ', CAST(age AS STRING), ', ',
      'Life Stage: ', life_stage, ', ',
      'Home Type: ', home_type, ', ',
      'Family Size: ', CAST(family_size AS STRING), ', ',
      'Work From Home: ', CASE WHEN work_from_home_flag THEN 'Yes' ELSE 'No' END, ', ',
      'Home Ownership: ', home_ownership, ', ',
      'Location: ', city, ', Postal Code: ', postal_code, '. ',
      
      'CUSTOMER ISSUE: ', issue_description, '. ',
      
      'ANALYSIS REQUIRED: ',
      '1. Device name (doorbell, camera, sensor, keypad, lock, etc.) ',
      '2. Is this Hardware or Software issue? ',
      '3. Onsite visit required: TRUE if technician needed, FALSE if customer can fix with guidance ',
      '4. Urgency: Measure the urgency on a scale of 1-5 with 5 being the highest'
      '5. TECHNICIAN REPORT (if onsite visit TRUE): ',
      '   Include customer context - Customer is ', CAST(age AS STRING), ' years old, ',
      life_stage, ', located in ', city, ' (Postal: ', postal_code, '). ',
      home_type, ' home with ', CASE WHEN work_from_home_flag THEN 'work from home setup' ELSE 'traditional work arrangement' END, '. ',
      '   Then provide: Issue verification, Severity (Low/Medium/High), Required tools and parts, Step-by-step repair actions, Estimated time ',
      
      '6. DIY INSTRUCTIONS (if onsite visit FALSE): ',
      '   Personalize for this customer profile - Consider they are ', life_stage, 
      CASE WHEN work_from_home_flag THEN ' working from home' ELSE '' END, 
      ' in a ', home_type, '. ',
      '   Provide simple, numbered steps they can follow during a guided support call. ',
      '   Make instructions appropriate for their technical comfort level.',
      image_ref
    ) AS prompt,
    ticket_id,
    customer_id,
    technician_id,
    issue_description,
    gcs_uri
    FROM `trilink-472019.database.active_customer_tickets_mm`
  ),
  STRUCT(
    "device_name STRING, hardware_or_software STRING,need_on_site_visit BOOL,urgency INT64, technician_report STRING ,diy_instructions STRING" AS output_schema
  )
));

Query is running:   0%|          |

""


Let's look at the table created a look at few sample cases

In [32]:
%%bigquery tech_soln

select * from `trilink-472019.database.active_customer_tickets_solutions` order by ticket_id;

Query is running:   0%|          |

Downloading:   0%|          |

In [33]:
##Display the table
tech_soln['hardware_or_software']=tech_soln['hardware_or_software'].str.lower()
tech_soln['has_tech_report'] = tech_soln['technician_report'].apply(lambda x: 0 if pd.isna(x) or x == '' else 1)
tech_soln['has_diy_instruction'] = tech_soln['diy_instructions'].apply(lambda x: 0 if pd.isna(x) or x == '' else 1)

tech_soln

,ticket_id,issue_description,device_name,hardware_or_software,need_on_site_visit,urgency,technician_report,diy_instructions,gcs_uri,has_tech_report,has_diy_instruction
0,TICKET_001,My main Security Panel is beeping every minute...,Security Panel (TriLink),hardware,True,4,"Customer is 64 years old, Empty_Nest, located ...",,gs://trilink/ticket-images/TICKET_001.jpeg,1,0
1,TICKET_002,The touchscreen on our Security Panel is compl...,Security Panel,hardware,True,5,"Customer is 46 years old, part of a Young_Fami...",,gs://trilink/ticket-images/TICKET_002.jpeg,1,0
2,TICKET_003,There's a 'Comm Fail' error on our security pa...,Security Panel/Keypad,hardware,True,5,"Customer is 57 years old, part of an Establish...",,gs://trilink/ticket-images/TICKET_003.jpeg,1,0
3,TICKET_004,My panel keeps losing AC power and then gettin...,Security System Panel Power Supply,hardware,False,3,,Hello! We understand that a constantly beeping...,gs://trilink/ticket-images/TICKET_004.jpeg,0,1
4,TICKET_005,I can't arm my system. The panel says 'Device ...,security system,hardware,True,5,"Customer (58, Empty_Nest, working from home in...",,gs://trilink/ticket-images/TICKET_005.jpeg,1,0
...,...,...,...,...,...,...,...,...,...,...,...
95,TICKET_096,My doorbell transformer is in the attic and I ...,Doorbell transformer,hardware,True,3,"Customer context: Customer is 45 years old, pa...",,gs://trilink/ticket-images/TICKET_096.jpeg,1,0
96,TICKET_097,The indoor camera keeps detecting the reflecti...,camera,software,False,2,,Hello! We understand that false motion alerts ...,gs://trilink/ticket-images/TICKET_097.jpeg,0,1
97,TICKET_098,My doorbell is not sending any motion alerts a...,doorbell,software,False,3,null,Hello! We understand it's important for your f...,gs://trilink/ticket-images/TICKET_098.jpeg,1,1
98,TICKET_099,The plastic mount for my indoor camera broke. ...,indoor camera,hardware,False,2,,Hello! We understand that a reliable indoor ca...,gs://trilink/ticket-images/TICKET_099.jpeg,0,1


Summary analysis == Let's look at:

- Freq of tickets that needs a technician
- Freq of issue type (hardware/software)
- Freq of Urgency

Then let's look at few cases and see what the issue was, solution deemed and correspodning reports made


In [34]:
for col in ['need_on_site_visit', 'hardware_or_software', 'urgency','has_tech_report','has_diy_instruction']:
    print(f"\n{col}:")
    print(tech_soln.groupby(col).agg({'ticket_id':'count'}))


need_on_site_visit:
                    ticket_id
need_on_site_visit           
False                      62
True                       36

hardware_or_software:
                      ticket_id
hardware_or_software           
hardware                     63
service quality               1
software                     34

urgency:
         ticket_id
urgency           
1                3
2                7
3               35
4               38
5               15

has_tech_report:
                 ticket_id
has_tech_report           
0                       45
1                       55

has_diy_instruction:
                     ticket_id
has_diy_instruction           
0                           27
1                           73


- We see some cases where the DIY and Tech report is created. However the usage of them depends on the need_on_site_visit flag. Business can double check these manually if needed to, but the urgency and need_on_site_visit should help business prioritize what tikets to assign to techniciains on prority so their valuable time can be used to optimal maximum

- Let's see some sample customer cases now

In [39]:
tickt_id = 'TICKET_070'
xdf = tech_soln[tech_soln['ticket_id'] == tickt_id].iloc[0]

print(f"🎫 Ticket: {tickt_id}")
print(f"📋 Issue: {xdf['issue_description']}")
print(f"🔧 Device: {xdf['device_name']} | Type: {xdf['hardware_or_software']}")
print(f"🚨 Urgency: {xdf['urgency']}/5 | Onsite Visit Needed: {'Yes' if xdf['need_on_site_visit'] == True else 'No'}\n")

# Display image
gcs_uri = xdf['gcs_uri']
public_url = gcs_uri.replace('gs://', 'https://storage.googleapis.com/')
display(Image(url=public_url, width=500))

# Show resolution
if xdf['need_on_site_visit'] == True:
    print("\n👨‍🔧 TECHNICIAN REPORT:")
    print(xdf['technician_report'] if xdf['technician_report'] != 'null' else 'Not available')
else:
    print("\n🛠️ DIY INSTRUCTIONS:")
    print(xdf['diy_instructions'])

🎫 Ticket: TICKET_070
📋 Issue: There is a loud humming noise coming from my main security panel box in the closet. It just started today and I'm worried it's a fire hazard.
🔧 Device: Security Panel | Type: hardware
🚨 Urgency: 5/5 | Onsite Visit Needed: Yes




👨‍🔧 TECHNICIAN REPORT:
Customer (44 years old, Established_Family, Single_Family home, 4 family members, not WFH, home owner in Toronto, M1S 0F1) reports a loud humming noise from the main security panel box in their closet, starting today, and is concerned about a fire hazard. 

Issue verification: The humming noise likely originates from the transformer or power supply circuit within the security panel, as indicated by the presence of a transformer in the provided image. This could signify an electrical fault or component failure. 
Severity: High. A loud hum from an electrical component, especially with a stated fire hazard concern, warrants immediate professional attention. 
Required tools and parts: Multimeter, insulated screwdriver set, wire strippers, potentially a replacement power supply board or transformer for the security panel. 
Step-by-step repair actions: 1. Safely de-energize the security panel by locating and turning off the corresponding circuit breaker. 2. Visually i

In [40]:
tickt_id = 'TICKET_081'
xdf = tech_soln[tech_soln['ticket_id'] == tickt_id].iloc[0]

print(f"🎫 Ticket: {tickt_id}")
print(f"📋 Issue: {xdf['issue_description']}")
print(f"🔧 Device: {xdf['device_name']} | Type: {xdf['hardware_or_software']}")
print(f"🚨 Urgency: {xdf['urgency']}/5 | Onsite Visit Needed: {'Yes' if xdf['need_on_site_visit'] == True else 'No'}\n")

# Display image
gcs_uri = xdf['gcs_uri']
public_url = gcs_uri.replace('gs://', 'https://storage.googleapis.com/')
display(Image(url=public_url, width=500))

# Show resolution
if xdf['need_on_site_visit'] == True:
    print("\n👨‍🔧 TECHNICIAN REPORT:")
    print(xdf['technician_report'] if xdf['technician_report'] != 'null' else 'Not available')
else:
    print("\n🛠️ DIY INSTRUCTIONS:")
    print(xdf['diy_instructions'])

🎫 Ticket: TICKET_081
📋 Issue: My doorbell camera is recording clips, but there is no sound at all. I have checked all the volume settings.
🔧 Device: doorbell camera | Type: software
🚨 Urgency: 3/5 | Onsite Visit Needed: No




🛠️ DIY INSTRUCTIONS:
Hello! We understand you're experiencing issues with the audio on your doorbell camera recordings. As an Established_Family in your Toronto home, it's important to have all your security features working correctly. It looks like the audio might simply be disabled in the app settings. Here are some steps you can follow during our call to enable it: 1. Open the Trillink app on your smartphone. 2. Navigate to the Live View of your Front Door Camera. 3. Look for a gear icon (⚙️) or settings icon, often located near the 'Audio Disabled' text or at the bottom right of the video player. 4. Tap on this settings icon to access the camera's specific settings. 5. Within the camera settings, locate an option related to 'Audio Recording', 'Microphone', or 'Sound'. 6. Ensure that this option is toggled 'On' or 'Enabled'. It might currently be set to 'Off' or 'Disabled'. 7. Save any changes if prompted. 8. Exit the settings and test your doorbell camera by recording a new clip o

In [41]:
tickt_id = 'TICKET_005'
xdf = tech_soln[tech_soln['ticket_id'] == tickt_id].iloc[0]

print(f"🎫 Ticket: {tickt_id}")
print(f"📋 Issue: {xdf['issue_description']}")
print(f"🔧 Device: {xdf['device_name']} | Type: {xdf['hardware_or_software']}")
print(f"🚨 Urgency: {xdf['urgency']}/5 | Onsite Visit Needed: {'Yes' if xdf['need_on_site_visit'] == True else 'No'}\n")

# Display image
gcs_uri = xdf['gcs_uri']
public_url = gcs_uri.replace('gs://', 'https://storage.googleapis.com/')
display(Image(url=public_url, width=500))

# Show resolution
if xdf['need_on_site_visit'] == True:
    print("\n👨‍🔧 TECHNICIAN REPORT:")
    print(xdf['technician_report'] if xdf['technician_report'] != 'null' else 'Not available')
else:
    print("\n🛠️ DIY INSTRUCTIONS:")
    print(xdf['diy_instructions'])

🎫 Ticket: TICKET_005
📋 Issue: I can't arm my system. The panel says 'Device Fault' but doesn't tell me which sensor is the problem. All my doors and windows are closed. This just started happening today.
🔧 Device: security system | Type: hardware
🚨 Urgency: 5/5 | Onsite Visit Needed: Yes




👨‍🔧 TECHNICIAN REPORT:
Customer (58, Empty_Nest, working from home in a Toronto Townhouse, Postal: M6G 0L8) is unable to arm their security system due to a 'Device Fault' message displayed on the panel/app. The system does not specify which sensor is causing the problem, despite the customer confirming all doors and windows are closed. This issue began today.

Issue verification: Confirm 'Device Fault' message and inability to arm the system. Attempt to access the system's diagnostic menu or fault log to identify the specific faulty device. Perform a full walk-test of all zones if system diagnostics are unspecific.
Severity: Medium (Security system compromised).
Required tools and parts: Security system diagnostic tool, multimeter, various sensor batteries (e.g., CR2032, AA, AAA, 12V), common replacement sensors (door/window contact, motion detector), small electronics screwdriver set.
Step-by-step repair actions:1. Verify the 'Device Fault' message on the customer's panel/app and con

In [42]:
tickt_id = 'TICKET_056'
xdf = tech_soln[tech_soln['ticket_id'] == tickt_id].iloc[0]

print(f"🎫 Ticket: {tickt_id}")
print(f"📋 Issue: {xdf['issue_description']}")
print(f"🔧 Device: {xdf['device_name']} | Type: {xdf['hardware_or_software']}")
print(f"🚨 Urgency: {xdf['urgency']}/5 | Onsite Visit Needed: {'Yes' if xdf['need_on_site_visit'] == True else 'No'}\n")

# Display image
gcs_uri = xdf['gcs_uri']
public_url = gcs_uri.replace('gs://', 'https://storage.googleapis.com/')
display(Image(url=public_url, width=500))

# Show resolution
if xdf['need_on_site_visit'] == True:
    print("\n👨‍🔧 TECHNICIAN REPORT:")
    print(xdf['technician_report'] if xdf['technician_report'] != 'null' else 'Not available')
else:
    print("\n🛠️ DIY INSTRUCTIONS:")
    print(xdf['diy_instructions'])

🎫 Ticket: TICKET_056
📋 Issue: I pressed the 'Test' button on my smoke alarm and it's very quiet. The sound is muffled and weak, not loud like it should be.
🔧 Device: smoke alarm | Type: hardware
🚨 Urgency: 5/5 | Onsite Visit Needed: No




🛠️ DIY INSTRUCTIONS:
Hello! It's understandable to be concerned about your smoke alarm not sounding properly, especially for your family's safety in your Toronto townhouse. Let's try some simple steps together to get it working as it should. Please take your time and follow these instructions carefully:
1.  Safety First: If your smoke alarm is on the ceiling, please ensure you have a stable step stool or ladder to reach it safely. You might find it helpful to have another family member assist you.
2.  Inspect for Obstructions: Carefully look at the vents and openings on the side of the smoke alarm. Sometimes dust, cobwebs, or even paint can partially block the speaker. If safe to do so, gently wipe around these openings with a dry cloth or use a soft brush to clear any visible debris.
3.  Replace Batteries: The most common reason for a weak or muffled alarm is low batteries. Even if the alarm chirps intermittently, a weak test sound often points to this.
    *   Gently twist or unlatc

In [43]:
tickt_id = 'TICKET_013'
xdf = tech_soln[tech_soln['ticket_id'] == tickt_id].iloc[0]

print(f"🎫 Ticket: {tickt_id}")
print(f"📋 Issue: {xdf['issue_description']}")
print(f"🔧 Device: {xdf['device_name']} | Type: {xdf['hardware_or_software']}")
print(f"🚨 Urgency: {xdf['urgency']}/5 | Onsite Visit Needed: {'Yes' if xdf['need_on_site_visit'] == True else 'No'}\n")

# Display image
gcs_uri = xdf['gcs_uri']
public_url = gcs_uri.replace('gs://', 'https://storage.googleapis.com/')
display(Image(url=public_url, width=500))

# Show resolution
if xdf['need_on_site_visit'] == True:
    print("\n👨‍🔧 TECHNICIAN REPORT:")
    print(xdf['technician_report'] if xdf['technician_report'] != 'null' else 'Not available')
else:
    print("\n🛠️ DIY INSTRUCTIONS:")
    print(xdf['diy_instructions'])

🎫 Ticket: TICKET_013
📋 Issue: There are small black dots or specks all over my indoor camera's video feed. I've wiped the outside of the lens but they're still there. It's like there is dust inside it.
🔧 Device: indoor camera | Type: hardware
🚨 Urgency: 4/5 | Onsite Visit Needed: Yes




👨‍🔧 TECHNICIAN REPORT:
Customer (40, Young_Family) located in a Single_Family home in Toronto (M2Z 5X4) with a work-from-home setup. Issue verification: The customer's indoor camera displays numerous small black dots/specks across the video feed, significantly obstructing the view. This indicates internal debris or dust within the camera's lens assembly, not externally cleanable. Severity: High, as the camera's primary function of clear monitoring is severely compromised. Required tools and parts: New Trilinik Indoor Cam unit, basic technician toolkit (screwdrivers, mounting hardware), ladder (if camera is mounted high). Step-by-step repair actions: 1. Arrive at the customer's Single_Family home. 2. Greet the customer and verify the reported issue by observing the camera's live feed, confirming the black specks. 3. Confirm that the issue is due to internal hardware failure/debris by noting no external damage or smudges. 4. Safely unmount the existing faulty Trilinik Indoor Cam. 5. Ins

#### Action 5 - BigQuery AI powered ADK agent as chatbot

- Idea:
    - We have a personalized emailing for retention, vector search for similar calls, best actions, agent coaching etc. created
    - These are jobs that can be scheduled for mass campaigns or weekly scheduled reports
    - However, we can also make this available as a real time assistant with same BigQuery tools added as tools to an ADK agent
    - For example, the call center agent can now:-
        - Get basic details about the customer and so initial approaches agent should follow at the start of the call
        - During the call can ask for next best actions given a scenario and can get it pulled up straght from business actions vector search from bigquery as tool
        - During the live call, pull up the customized personal message to make for the retention customer using similar bigquery AI.Generate ran for that customer
        - During live call Pull similar calls and how other agents handled it
        - Pull up the DIY ticket solution for a customer and walk them through it to solve a ticket on phone saving valuable techncian and customer's time
    - A Technician can now:-
        - Ask details about the work order scheduled by asking like "Give me the most urgent ticket to solve in Toronto that needs repair on security panel"
        - Ask about most similar techncial solutions to a problem using the vector search
    - Business:-
        - Can use BigQuery AI questions about customer base, current tickets, brainstorm on how to assign tickets efficiently etc among many things

In [2]:
from typing import Dict, List, Any, Optional
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
client = bigquery.Client(project="trilink-472019")
from queries.querystore import QueryStore
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.genai import types

import asyncio
import datetime
import requests
import os
import warnings
# Suppress warnings
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"
# Set only one API key to avoid the "Both GOOGLE_API_KEY and GEMINI_API_KEY" warning
if "GEMINI_API_KEY" in os.environ:
    del os.environ["GEMINI_API_KEY"]


qs=QueryStore()


In [3]:
def get_customer_personalized_retention_message(customer_id: str) -> Dict[str, Any]:
    """
    Generate a personalized retention script and problem analysis for a specific customer 
    using BigQuery AI capabilities.
    
    This function performs the following operations:
    1. Executes ML.EXPLAIN_PREDICT to analyze churn risk factors
    2. Uses VECTOR_SEARCH to find similar business problems and solutions
    3. Generates personalized retention script using AI.GENERATE
    4. Returns formatted results for agent use
    
    Args:
        customer_id (str): The unique customer identifier to analyze.
                          Must be a valid customer ID that exists in the database.
                          Example: "c123456"
    
    Returns:
        Dict[str, Any]: A dictionary containing:
            - success (bool): True if query executed successfully, False otherwise
            - retention_script (str): AI-generated personalized retention script for agent
            - customer_problem_identified (str): Analysis of customer's pain points and issues
            - error (str, optional): Error message if success=False
            - customer_id (str): The input customer ID for reference
    """
    try:
        query = qs.get_retention_call_script_mobile_query(customer_id=customer_id)
        query_job = client.query(query)
        df = query_job.to_dataframe()
        if df.empty:
            return {
                "success": False,
                "error": f"No data found for customer {customer_id}",
                "customer_id": customer_id
            }
        else:
            return {
                "success": True,
                "retention_script": str(df['live_retention_script'].iloc[0]),
                "customer_problem_identified": str(df["problem_analysis"].iloc[0]),
                "customer_churn_probability": df['churn_probability_percent'].iloc[0],
                "customer_id": customer_id
            }
    except Exception as e:
        return {
            "success": False,
            "error": f"Error processing customer {customer_id}: {str(e)}",
            "customer_id": customer_id
        }

In [4]:
def get_customer_info(customer_id: str) -> Dict[str, Any]:
    """
    Generates the basic details about the customer and so initial approaches agent should follow  
    using BigQuery AI capabilities.
    
    This function performs the following operations:
    1. Collects customers demographcis and all 3 service information if available
    2. Generates a quick Note for Agent using AI.GENERATE on gemini-2.5-flash-lite on 
        - Who the customer is
        - Personalized Conversation approach to follow
        - Key Talking points to discuss in case any delay on system processing or call hold is happenign
        - Potential cross/upsell opportunities available for customer 
    4. Returns formatted results for agent use
    
    Args:
        customer_id (str): The unique customer identifier to analyze.
                          Must be a valid customer ID that exists in the database.
                          Example: "c123456"
    
    Returns:
        Dict[str, Any]: A dictionary containing:
            - success (bool): True if query executed successfully, False otherwise
            - customer_info (str): AI-generated quick Note for Agent to use in his/her conversation with customer at start
            - error (str, optional): Error message if success=False
            - customer_id (str): The input customer ID for reference
    """
    try:
        query = qs.get_customer_info(customer_id=customer_id)
        query_job = client.query(query)
        df = query_job.to_dataframe()
        if df.empty:
            return {
                "success": False,
                "error": f"No data found for customer {customer_id}",
                "customer_id": customer_id
            }
        else:
            return {
                "success": True,
                "customer_info": str(df['customer_info'].iloc[0]),
                "customer_id": customer_id
            }
    except Exception as e:
        return {
            "success": False,
            "error": f"Error processing customer {customer_id}: {str(e)}",
            "customer_id": customer_id
        }

In [5]:
def get_business_solution_recommendations(problem_description: str):
    """
    Finds the closest matching business solutions based on problem description
    using vector similarity search
    
    This function performs the following operations:
    1. Agent can pass the problem customer is facing
    2. Vector search is done with the business solution vector store and the closest matching problem and its business solution is returned
    3. Root agent summarized and gives the best solution possible to agent
    
    Args:
        problem_description: The description of the problem the customer is facing that the agent is looking a solution for  
        
    Returns:
        Dictionary containing:
            - success: True if query executed successfully, False otherwise
            - problem_description: AI-generated quick Note for Agent to use in his/her conversation with customer at start
            - closest_matched_business_problem: The closest matched business problem from vector store
            - matched_problem: The business solution to the closest matched business problem to customers issue
            - error: Error message if success=False
    """
    try:
        query = qs.get_business_solution_recommendations(problem_description=problem_description)
        query_job = client.query(query)
        df = query_job.to_dataframe()
        if df.empty:
            return {
                "success": False,
                "error": f"No data found for problem_description {problem_description}"
            }
        else:
            return {
                "success": True,
                "problem_description": f"Customers Problem is: {problem_description}",
                "matched_problem": f"Closest Matched problem from business solution book is: {str(df['matched_problem'].iloc[0])}",
                "solution": f"Solution to this matched problem by business is: {str(df['recommended_solution'].iloc[0])}"
            }
    except Exception as e:
        return {
            "success": False,
            "error": f"Error processing customer {problem_description}: {str(e)}",
        }

In [6]:
def get_ticket_solution(customer_id: str) -> Dict[str, Any]:
    """
    Pulls up the solutions created for the tickets customer have made for their security devices.
    
    Will provide the resolution made for all the tickets and Step by step DIY instructions 
    if the issue was deemed to be solved on phone by agent.
    
    This function performs the following operations:
    1. Looks up the solutions for customer tickets created by customer_id
    2. Collects the resolution note and instructions to pass to customer for next actions
    3. If the resolution states a technician visit, convey the message to customer that 
       a technician will be coming shortly to solve their problem and they have all the 
       details of the issues
    4. If the resolution states "We will troubleshoot this on phone now", explain to 
       customer step by step how to solve their problem on phone
    
    Args:
        customer_id (str): The unique customer identifier to analyze.
                          Must be a valid customer ID that exists in the database.
                          Example: "c123456"
    
    Returns:
        Dict[str, Any]: A dictionary containing:
            - success (bool): True if query executed successfully, False otherwise
            - tickets (List[Dict]): List of ticket resolutions with details
            - ticket_count (int): Number of tickets found
            - customer_id (str): The input customer ID for reference
            - error (str, optional): Error message if success=False
    """
    try:
        query = qs.get_diy_solution(customer_id=customer_id)
        query_job = client.query(query)
        df = query_job.to_dataframe()
        
        if df.empty:
            return {
                "success": False,
                "error": f"No ticket resolutions found for customer {customer_id}",
                "customer_id": customer_id,
                "ticket_count": 0
            }
        
        # Convert DataFrame rows to list of dictionaries with structured information
        tickets = []
        for _, row in df.iterrows():
            ticket_info = {
                "issue_description": row.get('issue_description', ''),
                "device_name": row.get('device_name', ''),
                "hardware_or_software": row.get('hardware_or_software', ''),
                "urgency": row.get('urgency', ''),
                "resolution_mode": row.get('resolution_mode', ''),
                "diy_instructions": row.get('diy_instructions', '')
            }
            tickets.append(ticket_info)
        
        return {
            "success": True,
            "tickets": tickets,
            "ticket_count": len(tickets),
            "customer_id": customer_id,
            "summary": f"Found {len(tickets)} ticket resolution(s) for customer {customer_id}"
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": f"Error processing customer {customer_id}: {str(e)}",
            "customer_id": customer_id,
            "ticket_count": 0
        }

In [7]:
def get_tech_reports(tech_id: str, city: str) -> Dict[str, Any]:
    """
    Retrieves all active tickets assigned to a specific technician in a given city.
    
    This function pulls complete ticket information including issue descriptions,
    device details, urgency levels, and detailed technician reports for on-site visits.
    
    Args:
        tech_id (str): The unique technician identifier.
                      Example: "TECH001"
        city (str): The city where tickets are located.
                   Example: "Toronto"
    
    Returns:
        Dict[str, Any]: A dictionary containing:
            - success (bool): True if query executed successfully, False otherwise
            - tickets (List[Dict]): List of active tickets with full details
            - ticket_count (int): Number of tickets found
            - tech_id (str): The input technician ID for reference
            - city (str): The input city for reference
            - urgency_summary (Dict): Count of tickets by urgency level
            - error (str, optional): Error message if success=False
    """
    try:
        query = qs.get_tech_reports(tech_id=tech_id, city=city)
        query_job = client.query(query)
        df = query_job.to_dataframe()
        
        if df.empty:
            return {
                "success": False,
                "error": f"No active tickets found for technician {tech_id} in {city}",
                "tech_id": tech_id,
                "city": city,
                "ticket_count": 0
            }
        
        # Convert DataFrame rows to list of dictionaries with structured information
        tickets = []
        urgency_counts = {}
        
        for _, row in df.iterrows():
            urgency = str(row.get('urgency', ''))
            urgency_counts[urgency] = urgency_counts.get(urgency, 0) + 1
            
            ticket_info = {
                "ticket_id": row.get('ticket_id', ''),
                "device_name": row.get('device_name', ''),
                "hardware_or_software": row.get('hardware_or_software', ''),
                "urgency": urgency,
                "issue_description": row.get('issue_description', ''),
                "technician_report": row.get('technician_report', ''),
                "city": row.get('city', ''),
                "postal_code": row.get('postal_code', '')
            }
            tickets.append(ticket_info)
        
        # Sort tickets by urgency (highest first)
        tickets.sort(key=lambda x: int(x['urgency']) if x['urgency'].isdigit() else 0, reverse=True)
        
        return {
            "success": True,
            "tickets": tickets,
            "ticket_count": len(tickets),
            "tech_id": tech_id,
            "city": city,
            "urgency_summary": urgency_counts,
            "summary": f"Found {len(tickets)} active ticket(s) for technician {tech_id} in {city}"
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": f"Error retrieving tickets for technician {tech_id} in {city}: {str(e)}",
            "tech_id": tech_id,
            "city": city,
            "ticket_count": 0
        }
        
def get_tech_solution(question: str) -> Dict[str, Any]:
    """
    Searches the knowledge base for similar technical issues and their solutions.
    
    This function uses vector similarity search to find the most relevant past issues
    and their solutions based on the technician's question. Returns detailed solution
    steps and technician notes from previous similar cases.
    
    Args:
        question (str): The technical question or problem description from the technician.
                       Should be a clear description of the issue they're facing.
                       Example: "Camera showing flickering lines on the screen"
    
    Returns:
        Dict[str, Any]: A dictionary containing:
            - success (bool): True if query executed successfully, False otherwise
            - solutions (List[Dict]): List of similar problems with solutions and notes
            - solution_count (int): Number of relevant solutions found
            - question (str): The input question for reference
            - has_relevant_solutions (bool): True if distance <= 0.3 for any result
            - best_match_distance (float, optional): Distance score of best match
            - error (str, optional): Error message if success=False
    """
    try:
        query = qs.get_tech_solution(question=f"'{question}'")
        query_job = client.query(query)
        df = query_job.to_dataframe()
        
        if df.empty:
            return {
                "success": True,
                "solutions": [],
                "solution_count": 0,
                "question": question,
                "has_relevant_solutions": False,
                "message": "No highly relevant solutions found in knowledge base. This appears to be a new or unique issue."
            }
        
        # Convert DataFrame rows to list of dictionaries with structured information
        solutions = []
        
        for _, row in df.iterrows():
            solution_info = {
                "matched_problem": row.get('matched_problem', ''),
                "recommended_solution": row.get('recommended_solution', ''),
                "technician_notes": row.get('technician_notes', ''),
                "similarity_distance": float(row.get('distance', 1.0))
            }
            solutions.append(solution_info)
        
        # Sort by distance (closest match first)
        solutions.sort(key=lambda x: x['similarity_distance'])
        
        best_distance = solutions[0]['similarity_distance'] if solutions else 1.0
        
        return {
            "success": True,
            "solutions": solutions,
            "solution_count": len(solutions),
            "question": question,
            "has_relevant_solutions": best_distance <= 0.3,
            "best_match_distance": best_distance,
            "summary": f"Found {len(solutions)} similar case(s) in knowledge base"
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": f"Error searching knowledge base for '{question}': {str(e)}",
            "question": question,
            "solution_count": 0,
            "has_relevant_solutions": False
        }

In [8]:

# Define the subagents agent
agent_assistant = LlmAgent(
    name="agent_assistant",
    model="gemini-2.5-flash",
    instruction= """You are an agent assitant at TriLink. Your job is to help the agent find answers to their questions using the appropriate tools you have access to.
    If you don't know or don't have the tools to answer, say Sorry I can't help with that
    
    
    **Tool Usage Guidelines:**
    
    - If agent asks for a customer summary/notes/who the customer is  - use get_customer_info tool
    - If agent needs a retention script to follow or agent needs help to prevent churn for a customer, use get_customer_personalized_retention_message tool
    - If agent asks for help with an issue or look for what's the ideal businesss solution for a problem 
        - use get_business_solution_recommendations tool to get the matched problem and it's solution
        - Then analyze the actual customer issue,similarity of that to matched problem, who the customer is and what their plans/services are etc if it's available
        - Then create a personalized solution to offer for customers issue
        
     - **If agent asks about existing tickets, ticket resolutions, DIY instructions, or help with previously reported issues - use get_ticket_solution tool**
    
    **When using get_ticket_solution:**
    1. The tool will return multiple tickets if they exist
    2. For EACH ticket, check the resolution_mode field:
    
       **If resolution_mode = "Technician will visit site soon":**
       - Inform the customer that a technician has been scheduled to visit their location
       - Assure them the technician has all the details about their device_name issue
       - Let them know the technician will resolve the problem professionally
       - Ask if they have any questions while they wait
       
       **If resolution_mode = "We will troubleshoot this on phone now":**
       - Tell the customer you can help them resolve this issue right now over the phone
       - Present the DIY instructions step-by-step in a clear, easy-to-follow format
       - After presenting the instructions, ask: "Would you like me to walk you through these steps one at a time?"
       - If they say yes, guide them through each step, waiting for confirmation before moving to the next
       - If they say no, ask if they need clarification on any specific step
       - Check if the solution worked and if they need any additional help
    
    3. If multiple tickets are returned, address each one separately and ask which issue they'd like to focus on first
    
    **Communication Style:**
    - Be warm, patient, and reassuring
    - Use simple, non-technical language when explaining DIY steps
    - Break complex instructions into smaller, manageable steps
    - Always confirm understanding before moving forward
    - Show empathy for their frustration or concern
    """,
    
    description="Agent Assitant",
    tools=[
        FunctionTool(get_customer_personalized_retention_message),
        FunctionTool(get_customer_info),
        FunctionTool(get_business_solution_recommendations),
        FunctionTool(get_ticket_solution)
    ]
)


technician_assistant = LlmAgent(
    name="technician_assistant",
    model="gemini-2.5-flash",
    instruction= """You are a technician assistant at TriLink. Your job is to help technicians find answers to their questions using the appropriate tools you have access to.
    If you don't know or don't have the tools to answer, say Sorry I can't help with that.
    
    **Tool Usage Guidelines:**
    
    - **If technician asks for details on tickets assigned to them** - use get_tech_reports tool
    
    - **If technician asks a technical question or needs help troubleshooting an issue** - use get_tech_solution tool
    
    **When using get_tech_reports tool:**
    1. The tool will return multiple tickets if there are any assigned to the technician
    2. Always present tickets in order of urgency (highest urgency first)
    3. Format the output as follows:
    
       **Brief Overview:** 
       Provide a one-sentence summary of each ticket type/device issue
       
       Then for each ticket, present:
       
       **Ticket [ticket_id]:**
       - **Device Affected:** [device_name]
       - **Urgency Level:** [urgency]/5
       - **Location:** [city], [postal_code]
       - **Issue Description:** [issue_description]
       - **Technician Report:** 
         [Present the complete technician report exactly as provided - do not modify or summarize]
       
       ---
    
    **When using get_tech_solution tool:**
    1. The tool searches the knowledge base for similar past issues and solutions
    2. Check the has_relevant_solutions field:
       
       **If has_relevant_solutions = True:**
       - Analyze the matched_problem, recommended_solution, and technician_notes from the results
       - Consider the similarity distance (lower is better, <=0.3 is highly relevant)
       - Synthesize the information to provide detailed next step actions
       - Format your response as:
         
         **Knowledge Base Match Found:**
         - **Similar Issue:** [matched_problem]
         - **Similarity Score:** [show how close the match is]
         - **Recommended Solution Steps:**
           [Present the solution steps clearly]
         - **Technician Notes:**
           [Include important notes and tips from past cases]
         - **Suggested Next Actions:**
           [Based on the retrieved information, provide specific next steps]
       
       **If has_relevant_solutions = False:**
       - Inform the technician: "This appears to be a new or unique issue that we don't have highly relevant information for in our knowledge base."
       - Suggest: "Please consult the device manual or contact technical support for guidance."
       - If there are partial matches (distance > 0.3), you can mention them but note they may not be directly applicable
    
    **Important Guidelines:**
    - Never modify or summarize technician reports from get_tech_reports - present them exactly as they appear
    - For technical solutions, provide actionable next steps based on the knowledge base results
    - Always prioritize safety and proper procedures
    - If multiple similar solutions are found, synthesize the best practices from all of them
    """,
    
    description="Technician Assistant",
    tools=[
        FunctionTool(get_tech_reports),
        FunctionTool(get_tech_solution)
    ]
)

# Define the main agent
trilink_agents = LlmAgent(
    name="trilink_assistant",
    model="gemini-2.5-flash",
    instruction= """You are an assitant at TriLink. Your job is to help the user find answers to their questions by routing the question to appropriate subagents.
    If you don't know or don't have the subagents to answer, say Sorry I can't help with that
    
    
    **Sub Agent Routing Guidelines:**
    
    - If the user is an agent (callcenter) route to agent_assistant subagent
    - If the user is a technician route to technician_assitant subagent

    """,
    
    description="TriLink AI Assitant",
    sub_agents=[agent_assistant,technician_assistant]
)

In [9]:
# Constants
APP_NAME = "trilink_app"
USER_ID = "user123"

# Create InMemoryRunner with BOTH agent and app_name
runner = InMemoryRunner(agent=trilink_agents, app_name=APP_NAME)

# Create a session using the runner's session service
session = await runner.session_service.create_session(
    app_name=APP_NAME,  # This must match the app_name used in InMemoryRunner
    user_id=USER_ID
)

PERSISTENT_SESSION_ID = session.id

async def chat_with_trilink_agent_persistent(query: str):
    """Use persistent session to maintain conversation history"""
    print(f"User: {query}")
    
    user_content = types.Content(
        role='user', 
        parts=[types.Part(text=query)]
    )
    
    final_response = ""
    async for event in runner.run_async(
        user_id=USER_ID, 
        session_id=PERSISTENT_SESSION_ID,  # Now this session exists
        new_message=user_content
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text
            
    print(f"TriLink Assistant: {final_response}")
    # return final_response

In [16]:
session

Session(id='38ad283f-d101-4e18-93a3-1e62f162dcbc', app_name='trilink_app', user_id='user123', state={}, events=[], last_update_time=1758164791.087976)

In [12]:
await chat_with_trilink_agent_persistent("I'm a customer agent. Tell me who this customer is : C00000025")

User: I'm a customer agent. Tell me who this customer is : C00000025


TriLink Assistant: This customer, C00000025, is a 70-year-old long-standing customer with active Mobile and Internet services. They live in an area with a very low crime rate.

Here's a quick note for your conversation with them:

**WHO THEY ARE:**
This is a long-standing, 70-year-old customer with active Mobile and Internet services. They reside in an area with a very low crime rate, suggesting safety is likely not a primary concern for their current household setup.

**CONVERSATION APPROACH:**
Begin with a warm greeting and acknowledge their loyalty. Inquire about their current experience with their services to naturally transition into potential value-adds or optimizations.

**KEY TALKING POINTS:**
*   **Mobile Plan Optimization:** Discuss their Unlimited Standard plan and potential savings, especially if they have experienced data overages in the past.
*   **Internet Speed Experience:** Ask about their satisfaction with their 100 Mbps internet speed, particularly if they engage in 

In [39]:
await chat_with_trilink_agent_persistent("Customer experiencing chronic data insufficiency with Limited_2GB plan and multiple data overages")

User: Customer experiencing chronic data insufficiency with Limited_2GB plan and multiple data overages


TriLink Assistant: I understand your customer is experiencing chronic data insufficiency with their Limited_2GB plan and incurring multiple data overages.

The closest matching business problem identified is: "Customer with Limited_2GB plan hitting data limit by day 15 monthly, receiving $40-60 overage charges, working from home 3 days per week requiring video calls, considering competitor unlimited plans."

The recommended solution for this type of issue is to: "Upgrade to Unlimited_Standard $69/month eliminating overages and saving $25/month compared to current plan plus average overages. Bundle with home internet 100Mbps $49/month for reliable work-from-home setup. Offer first 3 months unlimited at 50% off and waive current overage charges as goodwill."

Given your customer's situation, I recommend offering them an upgrade to our **Unlimited Standard plan for $69/month**. This will eliminate their data overages entirely and, based on the typical overage charges, could save them appr

In [14]:
await chat_with_trilink_agent_persistent("Help me solve the tickets open for customer  : C00025032")

User: Help me solve the tickets open for customer  : C00025032


TriLink Assistant: I see that customer C00025032 has two open tickets.

For the issue with your **Security Panel**, where the touchscreen is frozen and unresponsive, a technician has already been scheduled to visit your site soon. They have all the details about the issue with your device and will resolve the problem professionally. Do you have any questions while you wait for their visit?

For the issue with your **Smoke Alarm**, where all interconnected alarms went off but you couldn't identify the one that started it, I can help you troubleshoot this over the phone right now. Here are the step-by-step instructions to identify which smoke alarm initiated the alert:

1.  **Look for the unique light:** Most modern interconnected smoke alarms will show a rapidly flashing red light or a steady red light on the specific alarm that first detected the issue. Walk through your home and observe each alarm carefully. The other alarms will usually flash differently or not at all, besides soundi

In [15]:
await chat_with_trilink_agent_persistent("Provide me the live agent retention callscript for customer id : C00000025")

User: Provide me the live agent retention callscript for customer id : C00000025


TriLink Assistant: Here is a live call retention script tailored for Mr./Ms. [Last Name]:

**(Agent Name):** "Thank you for calling [Company Name], this is (Agent Name). I see you've been a valued customer for over a year and a half, Mr./Ms. [Last Name], and we truly appreciate your loyalty. How can I help you today?"

**(Pause and listen actively for the customer's specific reason for calling. Show empathy.)**

**Agent:** "Thank you for sharing that, Mr./Ms. [Last Name]. My priority is to make sure you're always getting the best possible value and service from us. I’ve taken a look at your account, and I noticed you're currently on our Unlimited Standard mobile plan. While that's a great plan, I also see from your usage that you're not utilizing all the data that comes with it. Many of our senior customers find they can save a significant amount each month by switching to a plan that's a better fit for their actual needs."

**Agent:** "We have a special 'Senior Value' mobile plan spec

In [10]:
await chat_with_trilink_agent_persistent("I am technician. ID:tech002...Give me details of all the tickets open for me in toronto ")

User: I am technician. ID:tech002...Give me details of all the tickets open for me in toronto 


TriLink Assistant: **Brief Overview:**
There is a high-urgency security system issue where the panel displays 'Device Fault', and three medium-urgency camera-related issues: one requiring an outdoor camera to be repositioned due to an obstruction, another with an outdoor camera showing flickering video feed, and a video doorbell with a fogged-up lens.

**Ticket TICKET_005:**
- **Device Affected:** security system
- **Urgency Level:** 5/5
- **Location:** Toronto, M6G 0L8
- **Issue Description:** I can't arm my system. The panel says 'Device Fault' but doesn't tell me which sensor is the problem. All my doors and windows are closed. This just started happening today.
- **Technician Report:**
  Customer (58, Empty_Nest, working from home in a Toronto Townhouse, Postal: M6G 0L8) is unable to arm their security system due to a 'Device Fault' message displayed on the panel/app. The system does not specify which sensor is causing the problem, despite the customer confirming all doors and wind

In [11]:
await chat_with_trilink_agent_persistent(" Outdoor camera video feed exhibiting severe flickering and horizontal lines, consistent with customer report. Image quality is severely degraded, rendering the camera ineffective for security monitoring. What could be the possible solutions for this? ")

User:  Outdoor camera video feed exhibiting severe flickering and horizontal lines, consistent with customer report. Image quality is severely degraded, rendering the camera ineffective for security monitoring. What could be the possible solutions for this? 


TriLink Assistant: **Knowledge Base Match Found:**
- **Similar Issue:** trilink outdoor camera has a blurry or foggy image, especially at night.
- **Similarity Score:** 0.225
- **Recommended Solution Steps:**
  1. Cleaned the camera's outer lens dome with a microfiber cloth. The image remained foggy.
  2. Powered down the camera.
  3. Carefully unscrewed the outer dome/casing.
  4. Found moisture had gotten inside the housing, causing condensation on the inner lens.
  5. Used a lint-free cloth to gently dry the inside of the dome and the camera lens.
  6. Inspected the rubber gasket seal around the housing. It was cracked and brittle from sun exposure.
  7. Replaced the weatherproof gasket with a new one.
  8. Reassembled the camera housing, ensuring a tight seal.
  9. Powered the camera back on and verified the image was clear in both day and night (IR) mode.
- **Technician Notes:** Weather seals on outdoor cameras degrade over time. A compromised seal is the most common cause of inte